**МОНИТОРИНГ СОЦИАЛЬНЫХ ВЫПЛАТ**

In [2]:
import pandas as pd
import numpy as np
from datetime import date
import os
import sys
from IPython.display import clear_output
from geopy.distance import geodesic as GD
import difflib
import openpyxl as xl
import math
import re
import shutil # упаковка в архив
import warnings # игнорирование предупреждения о формате при импорте из excel

warnings.simplefilter("ignore")

# функции-хуюнкции
def lat(row):
    if row["r_latitude"] == "no_data" or row["r_longtitude"] == "no_data" or row["r_region_name"] == "no_data" or row["r_area"] == "no_data":
        if row["d_latitude"] == "no_data" or row["d_longtitude"] == "no_data" or row["d_region_name"] == "no_data" or row["d_area"] == "no_data":
            if row["mo_latitude"] == "no_data" or row["mo_longtitude"] == "no_data" or row["mo_region_name"] == "no_data" or row["mo_area"] == "no_data":
                return "no_data"
            else:
                return row["mo_latitude"]
        else:
            return row["d_latitude"]
    else:
        return row["r_latitude"]
    
def lon(row):
    if row["r_latitude"] == "no_data" or row["r_longtitude"] == "no_data" or row["r_region_name"] == "no_data" or row["r_area"] == "no_data":
        if row["d_latitude"] == "no_data" or row["d_longtitude"] == "no_data" or row["d_region_name"] == "no_data" or row["d_area"] == "no_data":
            if row["mo_latitude"] == "no_data" or row["mo_longtitude"] == "no_data" or row["mo_region_name"] == "no_data" or row["mo_area"] == "no_data":
                return "no_data"
            else:
                return row["mo_longtitude"]
        else:
            return row["d_longtitude"]
    else:
        return row["r_longtitude"]

def pref(row):
    if row["r_latitude"] == "no_data" or row["r_longtitude"] == "no_data" or row["r_region_name"] == "no_data" or row["r_area"] == "no_data":
        if row["d_latitude"] == "no_data" or row["d_longtitude"] == "no_data" or row["d_region_name"] == "no_data" or row["d_area"] == "no_data":
            if row["mo_latitude"] == "no_data" or row["mo_longtitude"] == "no_data" or row["mo_region_name"] == "no_data" or row["mo_area"] == "no_data":
                return "no_data"
            else:
                return row["mo_prefix"].lower()
        else:
            return row["d_prefix"].lower()
    else:
        return row["r_prefix"].lower()

def regn(row):
    if row["r_latitude"] == "no_data" or row["r_longtitude"] == "no_data" or row["r_region_name"] == "no_data" or row["r_area"] == "no_data":
        if row["d_latitude"] == "no_data" or row["d_longtitude"] == "no_data" or row["d_region_name"] == "no_data" or row["d_area"] == "no_data":
            if row["mo_latitude"] == "no_data" or row["mo_longtitude"] == "no_data" or row["mo_region_name"] == "no_data" or row["mo_area"] == "no_data":
                return "no_data"
            else:
                return row["mo_region_name"].lower()
        else:
            return row["d_region_name"].lower()
    else:
        return row["r_region_name"].lower()

def ara(row):
    if row["r_latitude"] == "no_data" or row["r_longtitude"] == "no_data" or row["r_region_name"] == "no_data" or row["r_area"] == "no_data":
        if row["d_latitude"] == "no_data" or row["d_longtitude"] == "no_data" or row["d_region_name"] == "no_data" or row["d_area"] == "no_data":
            if row["mo_latitude"] == "no_data" or row["mo_longtitude"] == "no_data" or row["mo_region_name"] == "no_data" or row["mo_area"] == "no_data":
                return "no_data"
            else:
                return row["mo_area"].lower()
        else:
            return row["d_area"].lower()
    else:
        return row["r_area"].lower()

def cut (coord):
    coord_0 = str(coord).replace(".", "")
    while len(coord_0) < 5:
        coord_0 += "0"
    return coord_0[:3]

def word_similarity(word1, word2):
    similarity = difflib.SequenceMatcher(None, word1, word2).ratio()
    return similarity


In [9]:
# пути к файлам и версии справочников (из названия файлов)
dir_path = os.getcwd()
dir_ = os.listdir(dir_path.replace("\\", "/"))
for file in dir_:
    if file.split("_")[0].split(".")[-1] == "1461":
        path_mo = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            mo_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
    if file.split("_")[0].split(".")[-1] == "114":
        path_dept = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            sp_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
    if file.split("_")[0].split(".")[-1] == "115":
        path_room = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            room_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
    if file.split("_")[0].split(".")[-1] == "1123":
        path_address = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            address_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
# path_save_sp = dir_path.replace("\\", "/") + "/sp.csv"
# path_save_room = dir_path.replace("\\", "/") + "/room.csv"
print("Версии справочников, извлечённые из названий файлов:")
print(
    f"\t- справочник МО: {mo_version}\n\t- справочник СП: {sp_version}\n\t- справочник кабинетов: {room_version}\n\t- справочник адресов: {address_version}"
)
if path_mo.split('.')[-1] == 'zip' or path_dept.split('.')[-2] == 'zip' or path_room.split('.')[-2] == 'zip':
    print('Скачанные архивы не распакованы, программа завершает работу.\nПопробуйте распаковать архивы.')
    sys.exit()

print('Загрузка справочников:')

df_mo = pd.read_excel(
    path_mo, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник МО")
df_dept = pd.read_excel(
    path_dept, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник СП")
df_room = pd.read_excel(
    path_room, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник кабинетов")
df_address = pd.read_excel(
    path_address, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник адресов")

Версии справочников, извлечённые из названий файлов:
	- справочник МО: 6.1261
	- справочник СП: 2.1619
	- справочник кабинетов: 2.1554
	- справочник адресов: 1.7
Загрузка справочников:
	- загружен справочник МО
	- загружен справочник СП
	- загружен справочник кабинетов
	- загружен справочник адресов


In [10]:
# локальные справочники
df_reg_dict = pd.read_csv("C:/work/scripts/справочники/social_benefits_reg.csv", sep=";")
df_reg_dict["reg_nsi"] = df_reg_dict["reg_nsi"].str.lower().str.replace(r'\W', '', regex=True)
df_reg_dict["reg_rosstat"] = df_reg_dict["reg_rosstat"].str.lower().str.replace(r'\W', '', regex=True)
df_clusters = pd.read_csv("C:/work/scripts/справочники/c&t.csv", sep=";")
df_clusters["region_name"] = df_clusters["region_name"].str.lower().str.replace(r'\W', '', regex=True)
df_clusters["pref"] = df_clusters["pref"].str.lower().str.replace(r'\W', '', regex=True)
df_clusters["area"] = df_clusters["area"].str.lower().str.replace(r'\W', '', regex=True)

In [11]:
# преобразование
df_room_s = (
    df_room[
        [
            "mo_oid",
            "depart_oid",
            "oid",
            "building_address_latitude",
            "building_address_longtitude",
            "building_address_region_name",
            "building_address_prefix_area",
            "building_address_area_name",
        ]
    ]
    .drop(index=0)
    .rename(columns={
        "oid": "room_oid",
        "building_address_latitude": "r_latitude",
        "building_address_longtitude": "r_longtitude",
        "building_address_region_name": "r_region_name",
        "building_address_prefix_area": "r_prefix",
        "building_address_area_name": "r_area",
        })
)
df_dept_s = (
    df_dept[
        [
            "depart_oid",
            "building_address_latitude",
            "building_address_longtitude",
            "building_address_region_name",
            "building_address_prefix_area",
            "building_address_area_name",
        ]
    ]
    .drop(index=0)
    .rename(
        columns={
            "building_address_latitude": "d_latitude",
            "building_address_longtitude": "d_longtitude",
            "building_address_region_name": "d_region_name",
            "building_address_prefix_area": "d_prefix",
            "building_address_area_name": "d_area",
        }
    )
)
df_mo_s = (
    df_mo[
        [
            "oid",
            "regionName",
            "latitude",
            "longtitude",
            "addrRegionName",
            "prefixArea",
            "areaName",
        ]
    ]
    .drop(index=0)
    .rename(columns={
        "oid": "mo_oid",
        "regionName": "mo_reg_name",
        "latitude": "mo_latitude",
        "longtitude": "mo_longtitude",
        "addrRegionName": "mo_region_name",
        "areaName": "mo_area",
        "prefixArea": "mo_prefix",
        })
)

In [12]:
%%time

# формирование адекватного адреса
df_adr = df_room_s.merge(df_mo_s, on="mo_oid", how="outer").merge(df_dept_s, on="depart_oid", how="outer")
df_adr = df_adr.fillna("no_data")
df_adr["lat"] = df_adr.apply(lat, axis=1)
df_adr["lon"] = df_adr.apply(lon, axis=1)
df_adr["reg"] = df_adr.apply(regn, axis=1).str.replace('город ', '').str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_adr["pref"] = df_adr.apply(pref, axis=1).str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_adr["area"] = df_adr.apply(ara, axis=1).str.replace('город ', '').str.replace(r'\W', '', regex=True).str.replace('ё', 'е')

df_adr_s = df_adr[["mo_oid", "depart_oid", "room_oid", "lat", "lon", "reg", "pref", "area"]]
df_adr_s = df_adr_s.merge(df_reg_dict, left_on="reg", right_on="reg_nsi", how="left")
df_adr_s.drop("reg_nsi", axis=1, inplace=True)
# объединение
# df_pclust = df_adr_s.merge(df_clusters, left_on=["reg_rosstat", "area"], right_on=["region_name", "area"], how="left")


CPU times: total: 8.52 s
Wall time: 40.2 s


In [7]:
# План "А", отвергнутый руководством
# df_pclust["pop_cluster"] = df_pclust["pop_cluster"].fillna(3)
# df_pclust["population"] = df_pclust["population"].fillna(0)
# df_pclust["pop_cluster"] = df_pclust["pop_cluster"].astype("int")
# df_pclust["population"] = df_pclust["population"].astype("int")
# df_pclust.loc[(df_pclust["reg"] == "москва")|(df_pclust["reg"] == "санктпетербург")|(df_pclust["reg"] == "севастополь"), "pop_cluster"] = 1
# df_pclust

**АНТОНОВ ОГОНЬ**   
(План "Б")

In [13]:
df_big_pop = pd.read_csv("C:/work/scripts/справочники/big_population.csv", sep=";", index_col=None)
df_big_pop["region"] = df_big_pop["region"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_big_pop["municipality"] = df_big_pop["municipality"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_big_pop["settlement"] = df_big_pop["settlement"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_big_pop["type"] = df_big_pop["type"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_big_pop = df_big_pop.drop("id", axis=1)

In [14]:
df_adr_s["lat"] = df_adr_s["lat"].str.replace("no_data", "0")
df_adr_s["lon"] = df_adr_s["lon"].str.replace("no_data", "0")
df_adr_s["lat"] = df_adr_s["lat"].astype("float")
df_adr_s["lon"] = df_adr_s["lon"].astype("float")
df_adr_s["region_found"] = ""
df_adr_s["municipality_found"] = ""
df_adr_s["settlement_found"] = ""
df_adr_s["settelment_type_found"] = ""
df_adr_s["population_found"] = ""
df_adr_s["children_found"] = ""
df_adr_s["D_km_found"] = ""


In [15]:
df_adr_s

,mo_oid,depart_oid,room_oid,lat,lon,reg,pref,area,reg_rosstat,region_found,municipality_found,settlement_found,settelment_type_found,population_found,children_found,D_km_found
0,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.26572,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,,,,,,,
1,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.26573,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,,,,,,,
2,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.26421,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,,,,,,,
3,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.33475,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,,,,,,,
4,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.33476,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964608,no_data,1.2.643.5.1.13.13.12.3.76.68776.0.667951,no_data,57.572804,39.926512,ярославскаяобласть,г,ярославль,ярославскаяобласть,,,,,,,
964609,no_data,1.2.643.5.1.13.13.12.3.76.68776.0.667942,no_data,57.568784,39.928102,ярославскаяобласть,г,ярославль,ярославскаяобласть,,,,,,,
964610,no_data,1.2.643.5.1.13.13.12.3.36.69446.0.691969,no_data,51.615566,39.238564,воронежскаяобласть,г,воронеж,воронежскаяобласть,,,,,,,
964611,no_data,1.2.643.5.1.13.13.12.3.26.68060.0.667658,no_data,44.061344,43.061408,ставропольскийкрай,г,пятигорск,ставропольскийкрай,,,,,,,


In [99]:
%%time

# аццкий код, поиск только с учётом названия населённого пункта
n = 0
lim = 500
for index, row in df_adr_s.iterrows():
    try:
        if index < lim:
            D_km = 100500
            region = "no_data"
            municipality = "no_data"
            settlement = "no_data"
            settelment_type = "no_data"
            population = "no_data"
            children = "no_data"
            pos_0 = (row["lat"], row["lon"])
            df_tmp = df_big_pop[df_big_pop["region"] == row["reg_rosstat"]]
            for i, r in df_tmp.iterrows():
                pos_tmp = (r["latitude_dd"], r["longitude_dd"])
                if row["area"] == r["settlement"]:
                    D_tmp = GD(pos_0, pos_tmp).km
                    if D_tmp < D_km:
                        region = r["region"]
                        municipality = r["municipality"]
                        settlement = r["settlement"]
                        settelment_type = r["type"]
                        population = r["population"]
                        children = r["children"]
                        D_km = D_tmp
            df_adr_s.at[index, "region_found"] = region
            df_adr_s.at[index, "municipality_found"] = municipality
            df_adr_s.at[index, "settlement_found"] = settlement
            df_adr_s.at[index, "settelment_type_found"] = settelment_type
            df_adr_s.at[index, "population_found"] = population
            df_adr_s.at[index, "children_found"] = children
            df_adr_s.at[index, "D_km_found"] = round(D_km, 3)
            n += 1
            clear_output()
            print(f"обработана строка {n} из {df_adr_s.shape[0]} при лимите {lim}\t\t{round(n*100/df_adr_s.shape[0], 3)} %", flush=True)
        else:
            break
    except Exception as e:
        print(f"строка {n} ошибка {e}")
print("сохранение")
df_adr_s.to_csv("C:/Users/Admin/Desktop/RESULT.csv", sep=";", encoding="utf-8-sig", index=False)
print("результат сохранён")

обработана строка 500 из 964951 при лимите 500		0.052 %
результат сохранён
CPU times: total: 8.22 s
Wall time: 37.7 s


In [71]:
%%time

# ещё более аццкий код-2, поиск с учётом названия населённого пункта, а если такого нет - только по координатам
n = 0
lim = 500
for index, row in df_adr_s.iterrows():
    try:
        if index < lim:
            D_km = 100500
            region = "no_data"
            municipality = "no_data"
            settlement = "no_data"
            settelment_type = "no_data"
            population = "no_data"
            children = "no_data"
            pos_0 = (row["lat"], row["lon"])
            df_tmp = df_big_pop[df_big_pop["region"] == row["reg_rosstat"]]
            if row["area"] in (df_tmp["settlement"].to_list()):
                for i, r in df_tmp.iterrows():
                    pos_tmp = (r["latitude_dd"], r["longitude_dd"])
                    if row["area"] == r["settlement"]:
                        D_tmp = GD(pos_0, pos_tmp).km
                        if D_tmp < D_km:
                            region = r["region"]
                            municipality = r["municipality"]
                            settlement = r["settlement"]
                            settelment_type = r["type"]
                            population = r["population"]
                            children = r["children"]
                            D_km = D_tmp
            else:
                for i, r in df_tmp.iterrows():
                    pos_tmp = (r["latitude_dd"], r["longitude_dd"])
                    D_tmp = GD(pos_0, pos_tmp).km
                    if D_tmp < D_km:
                            region = r["region"]
                            municipality = r["municipality"]
                            settlement = r["settlement"]
                            settelment_type = r["type"]
                            population = r["population"]
                            children = r["children"]
                            D_km = D_tmp
            df_adr_s.at[index, "region_found"] = region
            df_adr_s.at[index, "municipality_found"] = municipality
            df_adr_s.at[index, "settlement_found"] = settlement
            df_adr_s.at[index, "settelment_type_found"] = settelment_type
            df_adr_s.at[index, "population_found"] = population
            df_adr_s.at[index, "children_found"] = children
            df_adr_s.at[index, "D_km_found"] = round(D_km, 3)
            n += 1
            clear_output()
            print(f"обработана строка {n} из {df_adr_s.shape[0]} при лимите {lim}\t\t{round(n*100/df_adr_s.shape[0], 3)} %", flush=True)
        else:
            break
    except Exception as e:
        print(f"строка {n} ошибка {e}")
print("сохранение")
df_adr_s.to_csv("C:/Users/Admin/Desktop/RESULT_2.csv", sep=";", encoding="utf-8-sig", index=False)
print("результат сохранён")

обработана строка 500 из 963782 при лимите 500		0.052 %
результат сохранён
CPU times: total: 7.62 s
Wall time: 38.5 s


In [18]:
%%time

# совсем уже аццкий код-3, поиск с учётом названия населённого пункта, а если такого нет - только по координатам, статус поиска и совпадение букв

# необходимое только для кода-3 поле
df_adr_s["sf/e"] = ""


n = 0
lim = 20000
for index, row in df_adr_s.iterrows():
    sf = ""
    try:
        if index < lim:
            D_km = 100500
            region = "no_data"
            municipality = "no_data"
            settlement = "no_data"
            settelment_type = "no_data"
            population = "no_data"
            children = "no_data"
            pos_0 = (row["lat"], row["lon"])
            df_tmp = df_big_pop[df_big_pop["region"] == row["reg_rosstat"]]
            if row["area"] in (df_tmp["settlement"].to_list()):
                sf = "1"
                k = 0
                for i, r in df_tmp.iterrows():
                    pos_tmp = (r["latitude_dd"], r["longitude_dd"])
                    if row["area"] == r["settlement"]:
                        k += 1
                        D_tmp = GD(pos_0, pos_tmp).km
                        if D_tmp < D_km:
                            region = r["region"]
                            municipality = r["municipality"]
                            settlement = r["settlement"]
                            settelment_type = r["type"]
                            population = r["population"]
                            children = r["children"]
                            D_km = D_tmp
                sf = sf + "_" + str(k)
            else:
                sf = "0"
                l = 0
                for i, r in df_tmp.iterrows():
                    pos_tmp = (r["latitude_dd"], r["longitude_dd"])
                    D_tmp = GD(pos_0, pos_tmp).km
                    l += 1
                    if D_tmp < D_km:
                        region = r["region"]
                        municipality = r["municipality"]
                        settlement = r["settlement"]
                        settelment_type = r["type"]
                        population = r["population"]
                        children = r["children"]
                        D_km = D_tmp
                # срвнение названий населённых пунктов
                sf = sf + "_" + str(l) + "_" + str(round(word_similarity(row["area"], settlement), 2))
            df_adr_s.at[index, "region_found"] = region
            df_adr_s.at[index, "municipality_found"] = municipality
            df_adr_s.at[index, "settlement_found"] = settlement
            df_adr_s.at[index, "settelment_type_found"] = settelment_type
            df_adr_s.at[index, "population_found"] = population
            df_adr_s.at[index, "children_found"] = children
            df_adr_s.at[index, "D_km_found"] = round(D_km, 3)
            df_adr_s.at[index, "sf/e"] = sf
            n += 1
            clear_output()
            print(f"обработана строка {n} из {df_adr_s.shape[0]} при лимите {lim}\t{round(n*100/df_adr_s.shape[0], 3)} %\t({round(n*100/lim, 3)} %)", flush=True)
        else:
            break
    except Exception as e:
        print(f"строка {n} ошибка {e}")
        df_adr_s.at[index, "sf/e"] = e
print("сохранение")
df_adr_s.to_csv("C:/Users/Admin/Desktop/RESULT_3.csv", sep=";", encoding="utf-8-sig", index=False)
print("результат сохранён")

обработана строка 20000 из 964613 при лимите 20000	2.073 %	(100.0 %)
сохранение
результат сохранён
CPU times: total: 4min 50s
Wall time: 21min 13s


In [17]:
# df_adr_s.loc[df_adr_s["reg_rosstat"] == "москва", "population"] = 13104177
df_adr_s

,mo_oid,depart_oid,room_oid,lat,lon,reg,pref,area,reg_rosstat,region_found,municipality_found,settlement_found,settelment_type_found,population_found,children_found,D_km_found,sf/e
0,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.26572,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,кемеровскаяобласть,новокузнецкий,новокузнецк,г,551919,122291,0.695,1_1
1,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.26573,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,кемеровскаяобласть,новокузнецкий,новокузнецк,г,551919,122291,0.695,1_1
2,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.26421,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,кемеровскаяобласть,новокузнецкий,новокузнецк,г,551919,122291,0.695,1_1
3,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.33475,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,кемеровскаяобласть,новокузнецкий,новокузнецк,г,551919,122291,0.695,1_1
4,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.33476,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,кемеровскаяобласть,новокузнецкий,новокузнецк,г,551919,122291,0.695,1_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964608,no_data,1.2.643.5.1.13.13.12.3.76.68776.0.667951,no_data,57.572804,39.926512,ярославскаяобласть,г,ярославль,ярославскаяобласть,,,,,,,,
964609,no_data,1.2.643.5.1.13.13.12.3.76.68776.0.667942,no_data,57.568784,39.928102,ярославскаяобласть,г,ярославль,ярославскаяобласть,,,,,,,,
964610,no_data,1.2.643.5.1.13.13.12.3.36.69446.0.691969,no_data,51.615566,39.238564,воронежскаяобласть,г,воронеж,воронежскаяобласть,,,,,,,,
964611,no_data,1.2.643.5.1.13.13.12.3.26.68060.0.667658,no_data,44.061344,43.061408,ставропольскийкрай,г,пятигорск,ставропольскийкрай,,,,,,,,


In [28]:
df_adr_s[(df_adr_s["area"] == "мирный") & (df_adr_s["reg_rosstat"] == "саратовскаяобласть")]

,mo_oid,depart_oid,room_oid,lat,lon,reg,pref,area,reg_rosstat,region_found,municipality_found,settlement_found,settelment_type_found,population_found,children_found,D_km_found,sf/e
1941,1.2.643.5.1.13.13.12.2.64.6675,1.2.643.5.1.13.13.12.2.64.6675.0.175644,1.2.643.5.1.13.13.12.2.64.6675.0.175644.644904,52.219827,45.417680,саратовскаяобласть,п,мирный,саратовскаяобласть,саратовскаяобласть,петровскийрайон,мирный,п,321,73,0.477,1_4
237819,1.2.643.5.1.13.13.12.2.64.6563,1.2.643.5.1.13.13.12.2.64.6563.0.177662,1.2.643.5.1.13.13.12.2.64.6563.0.177662.639425,52.215617,48.889965,саратовскаяобласть,п,мирный,саратовскаяобласть,,,,,,,,
286954,1.2.643.5.1.13.13.12.2.64.6670,1.2.643.5.1.13.13.12.2.64.6670.0.524835,1.2.643.5.1.13.13.12.2.64.6670.0.524835.665054,50.908032,48.524207,саратовскаяобласть,п,мирный,саратовскаяобласть,,,,,,,,
286993,1.2.643.5.1.13.13.12.2.64.6670,1.2.643.5.1.13.13.12.2.64.6670.0.222278,1.2.643.5.1.13.13.12.2.64.6670.0.222278.664159,50.901682,48.522572,саратовскаяобласть,п,мирный,саратовскаяобласть,,,,,,,,
290696,1.2.643.5.1.13.13.12.2.64.6619,1.2.643.5.1.13.13.12.2.64.6619.0.174467,1.2.643.5.1.13.13.12.2.64.6619.0.174467.666510,50.352335,48.118294,саратовскаяобласть,п,мирный,саратовскаяобласть,,,,,,,,
406940,1.2.643.5.1.13.13.12.2.64.6621,1.2.643.5.1.13.13.12.2.64.6621.0.176164,1.2.643.5.1.13.13.12.2.64.6621.0.176164.289520,51.366284,48.660117,саратовскаяобласть,п,мирный,саратовскаяобласть,,,,,,,,


In [91]:
str = "ОЧЕНЬ ДЛИННОЕ НАИМЕНОВАНИЕ НЕКОТОРОГО МЕДИЦИНСКОГО УЧРЕЖДЕНИЯ"
str_list = str.split(" ")
new_str = " ".join(str_list[:2])
for element in str.split(" ")[2:-2]:
    element_short = " " + "".join([x for x in element][:3]) + "-" + "".join([x for x in element][-1])
    new_str += element_short
new_str = new_str + " " + str_list[-2] + " " + str_list[-1]
new_str

'ОЧЕНЬ ДЛИННОЕ НАИ-Е НЕК-О МЕДИЦИНСКОГО УЧРЕЖДЕНИЯ'

In [16]:
df_address.head(6)

,id,mo_region_name,mo_oid,mo_name_short,sp_oid,sp_name,fp_oid,fp_name,fp_type_name,bld_uuid,bld_id,bld_name,bld_address
0,ID записи справочника,Субъект РФ,OID медицинской организации,Краткое наименование медицинской организации,OID структурного подразделения,Наименование структурного подразделения,OID отделения/кабинета,Наименование отделения/кабинета,Наименование типа отделения/кабинета,GUID здания,ID здания,Наименование здания,Адрес здания
1,1,Свердловская область,1.2.643.5.1.13.13.12.2.66.22754,"ООО ""МКС ""БЕЛАЯ ЛАДЬЯ""",1.2.643.5.1.13.13.12.2.66.22754.0.460317,Фитнес центр,1.2.643.5.1.13.13.12.2.66.22754.0.460317.454771,NaN,Кабинет лечебной физкультуры,029075ac-3ca5-4bf7-8ef8-aa7d08307d90,163361,Фитнес центр,Свердловская Область г Екатеринбург ул Комсомо...
2,2,Республика Башкортостан,1.2.643.5.1.13.13.12.2.2.118,ГБУЗ РБ ГКБ № 5 Г.УФА,1.2.643.5.1.13.13.12.2.2.118.0.20566,Поликлиническое отделение 1 (Цюрупы 84),1.2.643.5.1.13.13.12.2.2.118.0.20566.15922,NaN,Кабинеты врачей гематологов,60152184-63e2-4cfb-8722-9d4bbbe7aa81,10943,Здание поликлиники (Цюрупы 84),Республика Башкортостан г Уфа ул Цюрупы 84
3,3,Кемеровская область,1.2.643.5.1.13.13.12.2.42.3939,ГАУЗ КО КККДЦ,1.2.643.5.1.13.13.12.2.42.3939.0.130687,Поликлиника № 1 Терапевтическое отделение 1,1.2.643.5.1.13.13.12.2.42.3939.0.130687.189120,NaN,Терапевтические,5be94957-5695-4262-abaa-94ec5c1e9cda,59179,ГАУЗ ККДЦ Поликлиника 1,Кемеровская область г Кемерово пр-кт Ленинград...
4,4,Архангельская область,1.2.643.5.1.13.13.12.2.29.2693,"ГБУЗ ""АГКБ № 7""",1.2.643.5.1.13.13.12.2.29.2693.0.63940,Педиатрическое отделение,1.2.643.5.1.13.13.12.2.29.2693.0.63940.55102,NaN,Педиатра участкового кабинет,c854d0bc-dafc-48d5-9129-1f9fc4c9f403,2527,нежилое помещение на ул Приморская,Архангельская Область г Архангельск ул Приморс...
5,5,Вологодская область,1.2.643.5.1.13.13.12.2.35.3271,"БУЗ ВО ""ТАРНОГСКАЯ ЦРБ""",1.2.643.5.1.13.13.12.2.35.3271.0.62159,Поликлиника,1.2.643.5.1.13.13.12.2.35.3271.0.62159.56147,NaN,Зубопротезные,98d839c1-b9a5-4c37-abcf-61757aefb5d6,32211,Здание больницы,Вологодская с Тарногский Городок ул Погранична...


In [21]:
import requests
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup
import re
from lxml import html
import time

In [22]:
site = "https://geoadm.com"
reg_dict = {
    "/naselennye-punkty-moskovskoy-oblasti.html": "Московская область",
    "/naselennye-punkty-krasnodarskogo-kraya.html": "Краснодарский край",
    "/naselennye-punkty-sverdlovskoy-oblasti.html": "Свердловская область",
    "/naselennye-punkty-rostovskoy-oblasti.html": "Ростовская область",
    "/naselennye-punkty-respubliki-bashkortostan.html": "Республика Башкортостан",
    "/naselennye-punkty-respubliki-tatarstan.html": "Республика Татарстан",
    "/naselennye-punkty-chelyabinskoy-oblasti.html": "Челябинская область",
    "/naselennye-punkty-nizhegorodskoy-oblasti.html": "Нижегородская область",
    "/naselennye-punkty-samarskoy-oblasti.html": "Самарская область",
    "/naselennye-punkty-respubliki-dagestan.html": "Республика Дагестан",
    "/naselennye-punkty-krasnoyarskogo-kraya.html": "Красноярский край",
    "/naselennye-punkty-stavropolskogo-kraya.html": "Ставропольский край",
    "/naselennye-punkty-novosibirskoy-oblasti.html": "Новосибирская область",
    "/naselennye-punkty-kemerovskoy-oblasti.html": "Кемеровская область",
    "/naselennye-punkty-permskogo-kraya.html": "Пермский край",
    "/naselennye-punkty-volgogradskoy-oblasti.html": "Волгоградская область",
    "/naselennye-punkty-saratovskoy-oblasti.html": "Саратовская область",
    "/naselennye-punkty-irkutskoy-oblasti.html": "Иркутская область",
    "/naselennye-punkty-voronezhskoy-oblasti.html": "Воронежская область",
    "/naselennye-punkty-altaiskogo-kraya.html": "Алтайский край",
    "/naselennye-punkty-orenburgskoy-oblasti.html": "Оренбургская область",
    "/naselennye-punkty-omskoy-oblasti.html": "Омская область",
    "/naselennye-punkty-respubliki-krym.html": "Республика Крым",
    "/naselennye-punkty-leningradskoy-oblasti.html": "Ленинградская область",
    "/naselennye-punkty-primorskogo-kraya.html": "Приморский край",
    "/naselennye-punkty-hanty-mansiyskogo-avtonomnogo-okruga.html": "Ханты-Мансийский автономный округ - Югра",
    "/naselennye-punkty-tyumenskoy-oblasti.html": "Тюменская область",
    "/naselennye-punkty-belgorodskoy-oblasti.html": "Белгородская область",
    "/naselennye-punkty-chechenskoy-respubliki.html": "Чеченская Республика",
    "/naselennye-punkty-udmurtskoy-respubliki.html": "Удмуртская Республика",
    "/naselennye-punkty-tulskoy-oblasti.html": "Тульская область",
    "/naselennye-punkty-vladimirskoy-oblasti.html": "Владимирская область",
    "/naselennye-punkty-habarovskogo-kraya.html": "Хабаровский край",
    "/naselennye-punkty-penzenskoy-oblasti.html": "Пензенская область",
    "/naselennye-punkty-kirovskoy-oblasti.html": "Кировская область",
    "/naselennye-punkty-tverskoy-oblasti.html": "Тверская область",
    "/naselennye-punkty-yaroslavskoy-oblasti.html": "Ярославская область",
    "/naselennye-punkty-ulianovskoy-oblasti.html": "Ульяновская область",
    "/naselennye-punkty-chuvashskoy-respubliki.html": "Чувашская Республика",
    "/naselennye-punkty-bryanskoy-oblasti.html": "Брянская область",
    "/naselennye-punkty-vologodskoy-oblasti.html": "Вологодская область",
    "/naselennye-punkty-lipetskoy-oblasti.html": "Липецкая область",
    "/naselennye-punkty-ryazanskoy-oblasti.html": "Рязанская область",
    "/naselennye-punkty-kurskoy-oblasti.html": "Курская область",
    "/naselennye-punkty-arhangelskoy-oblasti.html": "Архангельская область",
    "/naselennye-punkty-tomskoy-oblasti.html": "Томская область",
    "/naselennye-punkty-zabaikalskogo-kraya.html": "Забайкальский край",
    "/naselennye-punkty-kaliningradskoy-oblasti.html": "Калининградская область",
    "/naselennye-punkty-kaluzhskoy-oblasti.html": "Калужская область",
    "/naselennye-punkty-astrahanskoy-oblasti.html": "Астраханская область",
    "/naselennye-punkty-tambovskoy-oblasti.html": "Тамбовская область",
    "/naselennye-punkty-ivanovskoy-oblasti.html": "Ивановская область",
    "/naselennye-punkty-respubliki-buryatiya.html": "Республика Бурятия",
    "/naselennye-punkty-yakutii.html": "Республика Саха (Якутия)",
    "/naselennye-punkty-smolenskoy-oblasti.html": "Смоленская область",
    "/naselennye-punkty-kabardino-balkarskoy-respubliki.html": "Кабардино-Балкарская Республика",
    "/naselennye-punkty-kurganskoy-oblasti.html": "Курганская область",
    "/naselennye-punkty-respubliki-komi.html": "Республика Коми",
    "/naselennye-punkty-amurskoy-oblasti.html": "Амурская область",
    "/naselennye-punkty-respubliki-mordoviya.html": "Республика Мордовия",
    "/naselennye-punkty-murmanskoy-oblasti.html": "Мурманская область",
    "/naselennye-punkty-orlovskoy-oblasti.html": "Орловская область",
    "/naselennye-punkty-respubliki-severnaya-osetiya-alaniya.html": "Республика Северная Осетия - Алания",
    "/naselennye-punkty-respubliki-mariy-el.html": "Республика Марий Эл",
    "/naselennye-punkty-kostromskoy-oblasti.html": "Костромская область",
    "/naselennye-punkty-pskovskoy-oblasti.html": "Псковская область",
    "/naselennye-punkty-respubliki-kareliya.html": "Республика Карелия",
    "/naselennye-punkty-novgorodskoy-oblasti.html": "Новгородская область",
    "/naselennye-punkty-yamalo-nenetskogo-avtonomnogo-okruga.html": "Ямало-Ненецкий автономный округ",
    "/naselennye-punkty-respubliki-hakasiya.html": "Республика Хакасия",
    "/naselennye-punkty-respubliki-ingushetiya.html": "Республика Ингушетия",
    "/naselennye-punkty-sahalinskoy-oblasti.html": "Сахалинская область",
    "/naselennye-punkty-karachaevo-cherkesskoy-respubliki.html": "Карачаево-Черкесская Республика",
    "/naselennye-punkty-respubliki-adygeya.html": "Республика Адыгея",
    "/naselennye-punkty-respubliki-tyva.html": "Республика Тыва",
    "/naselennye-punkty-kamchatskogo-kraya.html": "Камчатский край",
    "/naselennye-punkty-respubliki-kalmykiya.html": "Республика Калмыкия",
    "/naselennye-punkty-respubliki-altay.html": "Республика Алтай",
    "/naselennye-punkty-evreiskoy-avtonomnoy-oblasti.html": "Еврейская автономная область",
    "/naselennye-punkty-magadanskoy-oblasti.html": "Магаданская область",
    "/naselennye-punkty-chukotskogo-avtonomniy-okruga.html": "Чукотский автономный округ",
    "/naselennye-punkty-nenetskogo-avtonomniy-okruga.html": "Ненецкий автономный округ",
}

In [23]:
%%time

df_new_population = pd.DataFrame()
for key, value in reg_dict.items():
    try:
        df_tmp = pd.read_html(site + key, index_col=None, header=0, encoding="utf-8")[1].drop("Unnamed: 0", axis=1)
        df_tmp.insert(0, "region", value)
        df_tmp.rename(columns={"Название": "area", "Тип": "prefix", "Население": "population", "Округ/Район": "municipality", "Округ/район": "municipality"},inplace=True)
        df_new_population = pd.concat([df_new_population, df_tmp])
        print(f"сделано {value}")
    except Exception as e:
        print(f"{key} трабл: {e}")
    # break
df_new_population = df_new_population.reset_index(drop=True)
df_new_population["population"] = df_new_population["population"].fillna(0)
df_new_population["population"] = df_new_population["population"].replace(r"\s+", "", regex=True)
df_new_population["population"] = df_new_population["population"].astype("int")
df_new_population

сделано Московская область
сделано Краснодарский край
сделано Свердловская область
сделано Ростовская область
сделано Республика Башкортостан
сделано Республика Татарстан
сделано Челябинская область
сделано Нижегородская область
сделано Самарская область
сделано Республика Дагестан
сделано Красноярский край
сделано Ставропольский край
сделано Новосибирская область
сделано Кемеровская область
сделано Пермский край
сделано Волгоградская область
сделано Саратовская область
сделано Иркутская область
сделано Воронежская область
сделано Алтайский край
сделано Оренбургская область
сделано Омская область
сделано Республика Крым
сделано Ленинградская область
сделано Приморский край
сделано Ханты-Мансийский автономный округ - Югра
сделано Тюменская область
сделано Белгородская область
сделано Чеченская Республика
сделано Удмуртская Республика
сделано Тульская область
сделано Владимирская область
сделано Хабаровский край
сделано Пензенская область
сделано Кировская область
сделано Тверская област

,region,area,prefix,population,municipality
0,Московская область,Балашиха,город,507307,Городской округ Балашиха
1,Московская область,Подольск,город,309250,Городской округ Подольск
2,Московская область,Химки,город,257757,Городской округ Химки
3,Московская область,Мытищи,город,245643,Городской округ Мытищи
4,Московская область,Королёв,город,225299,Городской округ Королёв
...,...,...,...,...,...
155266,Магаданская область,Беличан,пгт (рабочий посёлок),0,Сусуманский городской округ
155267,Магаданская область,Обо,посёлок,0,Тенькинский городской округ
155268,Магаданская область,Верхний Ат-Урях,пгт (рабочий посёлок),0,Ягоднинский городской округ
155269,Магаданская область,имени Горького,посёлок,0,Ягоднинский городской округ


In [10]:
df_new_pop = df_new_population.copy()
df_new_pop["_region"] = df_new_pop["region"].str.lower().str.replace(r'\W', '', regex=True)
df_new_pop["_area"] = df_new_pop["area"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_new_pop["_region"] = df_new_pop["_region"].str.replace('республикакрымa', 'республикакрым')
# df_new_pop.head(3)
df_new_pop_test = df_new_pop[["_region", "_area", "prefix"]]
df_new_pop_test

,_region,_area,prefix
0,московскаяобласть,балашиха,город
1,московскаяобласть,подольск,город
2,московскаяобласть,химки,город
3,московскаяобласть,мытищи,город
4,московскаяобласть,королев,город
...,...,...,...
155266,магаданскаяобласть,беличан,пгт (рабочий посёлок)
155267,магаданскаяобласть,обо,посёлок
155268,магаданскаяобласть,верхнийатурях,пгт (рабочий посёлок)
155269,магаданскаяобласть,именигорького,посёлок


In [11]:
df_big_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155921 entries, 0 to 155920
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   region         155921 non-null  object 
 1   municipality   155921 non-null  object 
 2   settlement     155921 non-null  object 
 3   type           155921 non-null  object 
 4   population     155921 non-null  int64  
 5   children       155921 non-null  int64  
 6   latitude_dms   155921 non-null  object 
 7   longitude_dms  155921 non-null  object 
 8   latitude_dd    155921 non-null  float64
 9   longitude_dd   155921 non-null  float64
 10  oktmo          155920 non-null  float64
 11  dadata         155921 non-null  int64  
 12  rosstat        155921 non-null  int64  
dtypes: float64(3), int64(4), object(6)
memory usage: 15.5+ MB


In [12]:
# df_big_pop.head(3)
df_big_pop_test = df_big_pop[["region", "settlement", "type"]]
df_big_pop_test["settlement"] = df_big_pop_test["settlement"].str.replace('ё', 'е')
df_big_pop_test["region"] = df_big_pop_test["region"].str.replace('республикакрыма', 'республикакрым')
df_big_pop_test

,region,settlement,type
0,орловскаяобласть,колонтаева,д
1,республикакрым,пушкино,с
2,липецкаяобласть,барятино,с
3,тверскаяобласть,хилово,д
4,томскаяобласть,басмасово,д
...,...,...,...
155916,ульяновскаяобласть,ребровка,д
155917,псковскаяобласть,шадрица,д
155918,псковскаяобласть,овчинниково,д
155919,тульскаяобласть,новогуровский,рп


In [17]:
df_test_0 = df_big_pop_test.merge(df_new_pop_test, left_on=["region", "settlement"], right_on=["_region", "_area"], how="outer")
df_test_na = df_test_0[df_test_0["region"].isna() | df_test_0["settlement"].isna() | df_test_0["_region"].isna() | df_test_0["_area"].isna()].reset_index(drop=True).drop_duplicates()
df_test_na

,region,settlement,type,_region,_area,prefix
0,псковскаяобласть,хибины,д,NaN,NaN,NaN
1,свердловскаяобласть,новыекривки,д,NaN,NaN,NaN
2,вологодскаяобласть,подлесное,д,NaN,NaN,NaN
3,республикадагестан,гавутли,с,NaN,NaN,NaN
4,свердловскаяобласть,унжепавинск,д,NaN,NaN,NaN
...,...,...,...,...,...,...
7692,NaN,NaN,NaN,магаданскаяобласть,беличан,пгт (рабочий посёлок)
7693,NaN,NaN,NaN,магаданскаяобласть,обо,посёлок
7694,NaN,NaN,NaN,магаданскаяобласть,верхнийатурях,пгт (рабочий посёлок)
7695,NaN,NaN,NaN,магаданскаяобласть,именигорького,посёлок


In [28]:
# численность населения
df_new_population
df_new_pop_group = pd.DataFrame(df_new_population.groupby("region")["population"].sum()).reset_index()
df_new_pop_group["region"] = df_new_pop_group["region"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_new_pop_group = df_new_pop_group.rename(columns = {"population": "pop_data_2"})
df_big_pop_group = pd.DataFrame(df_big_pop.groupby("region")["population"].sum()).reset_index()
df_big_pop_group = df_big_pop_group.rename(columns = {"population": "pop_data_1"})
df_new_pop_group

,region,pop_data_2
0,алтайскийкрай,2404826
1,амурскаяобласть,809257
2,архангельскаяобласть,1147490
3,астраханскаяобласть,999322
4,белгородскаяобласть,1564443
...,...,...
75,челябинскаяобласть,3492429
76,чеченскаяреспублика,1368233
77,чувашскаяреспублика,1284684
78,ямалоненецкийавтономныйокруг,535816


In [52]:
df_pop_group = df_big_pop_group.merge(df_new_pop_group, on="region", how="outer")
df_pop_group[["pop_data_1", "pop_data_2"]] = df_pop_group[["pop_data_1", "pop_data_2"]].fillna(0)
df_pop_group[["pop_data_1", "pop_data_2"]] = df_pop_group[["pop_data_1", "pop_data_2"]].astype("int")
df_pop_group["d_2/d_1(%)"] = round(df_pop_group["pop_data_2"]*100/df_pop_group["pop_data_1"], 1)
df_pop_group["d_2/d_1(%)"] = df_pop_group["d_2/d_1(%)"].fillna(0)

In [53]:
df_pop_group

,region,pop_data_1,pop_data_2,d_2/d_1(%)
0,алтайскийкрай,2332813,2404826,103.1
1,амурскаяобласть,786213,809257,102.9
2,архангельскаяобласть,1100247,1147490,104.3
3,астраханскаяобласть,1016036,999322,98.4
4,байконур,0,0,0.0
...,...,...,...,...
81,чеченскаяреспублика,1456949,1368233,93.9
82,чувашскаяреспублика,1221431,1284684,105.2
83,чукотскийавтономныйокруг,49790,0,0.0
84,ямалоненецкийавтономныйокруг,545045,535816,98.3


In [59]:
df_new_mean = pd.DataFrame(df_new_pop.groupby("prefix")["population"].mean()).reset_index()
df_big_mean = pd.DataFrame(df_big_pop.groupby("type")["population"].mean()).reset_index()

In [60]:
df_new_mean

,prefix,population
0,Город,28267.214286
1,ПГТ,3327.500000
2,ПГТ (дачный посёлок),6380.222222
3,ПГТ (курортный посёлок),2105.000000
4,аал,259.979167
...,...,...
89,станция,201.799270
90,улус,423.743169
91,усадьба,68.000000
92,участок,78.258621


In [61]:
df_big_mean

,type,population
0,c,48.250000
1,x,26.000000
2,аал,208.769231
3,автодорога,0.000000
4,арбан,116.000000
5,аул,1129.700508
6,высел,33.169811
7,г,86902.946087
8,городок,1875.714286
9,гп,3763.833333


In [62]:
df_big_pop[df_big_pop["type"] == "рн"]

,region,municipality,settlement,type,population,children,latitude_dms,longitude_dms,latitude_dd,longitude_dd,oktmo,dadata,rosstat
2103,астраханскаяобласть,городастрахань,ленинский,рн,148717,28855,46.23.40,048.06.24,46.394444,48.106667,1.270100e+07,1,0
6689,астраханскаяобласть,городастрахань,трусовский,рн,115208,23282,46.21.06,047.59.44,46.351667,47.995556,1.270100e+07,1,0
21811,санктпетербург,центральный,центральный,рн,0,0,59.55.36,030.21.31,59.926667,30.358611,4.034300e+07,1,0
26939,республикабашкортостан,уфа,октябрьский,рн,240708,44727,54.45.57,056.01.53,54.765833,56.031389,8.070100e+07,1,0
37700,красноярскийкрай,норильск,талнах,рн,47307,11204,69.29.21,088.24.24,69.489167,88.406667,4.729000e+06,1,0
37837,республикабашкортостан,уфа,советский,рн,177865,29691,54.44.13,055.59.10,54.736944,55.986111,8.070100e+07,1,0
41100,республикадагестан,махачкала,кировскийрайон,рн,0,0,42.59.30,047.25.04,42.991667,47.417778,8.270100e+10,1,0
45289,республикабашкортостан,уфа,кировский,рн,156820,31455,54.41.12,055.58.51,54.686667,55.980833,8.070100e+07,1,0
48669,республикабашкортостан,уфа,демский,рн,70365,14413,54.41.56,055.55.01,54.698889,55.916944,8.070100e+07,1,0
58184,республикабашкортостан,уфа,калининский,рн,199883,38010,54.45.51,056.09.39,54.764167,56.160833,8.070100e+07,1,0


***

**ПОПЫТКА (НЕУДАЧНАЯ) ПОИСКА НАЗВАНИЙ СП ПО НАЗВАНИЮ СП**

In [73]:
df_dept = df_dept.drop(index=0)
df_dept

,mo_oid,depart_oid,depart_create_date,depart_modify_date,depart_liquidation_date,depart_name,depart_type_id,depart_type_name,depart_kind_id,depart_kind_name,...,hospital_home_building_address_aoid_street,hospital_home_building_address_houseid,hospital_home_building_address_prefix_area,hospital_home_building_address_area_name,hospital_home_building_address_prefix_street,hospital_home_building_address_street_name,hospital_home_building_address_house,hospital_home_building_address_building,hospital_home_building_address_struct,hospital_home_building_address_fias_version
1,1.2.643.5.1.13.13.12.3.23.3371,1.2.643.5.1.13.13.12.3.23.3371.0.587129,2023-02-03T06:14:05.158+0300,2023-05-22T08:50:14.939+0300,NaN,"Саранск, Севастопольская, 56, к 2, помещение 2 А",7,Фармацевтический,1271,Аптечные пункты,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.2.643.5.1.13.13.12.2.46.4378,1.2.643.5.1.13.13.12.2.46.4378.0.399700,2020-07-08T17:39:40.212+0300,2022-08-23T23:05:19.972+0300,01.07.2020,Эндоскопический кабинет,4,Инструментально-диагностический,1197,Эндоскопии,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.2.643.5.1.13.13.12.2.55.11157,1.2.643.5.1.13.13.12.2.55.11157.0.381437,2020-03-20T09:24:52.108+0300,2022-08-23T23:04:02.662+0300,NaN,"Бюро № 3 филиал ФКУ ""ГБ МСЭ по Омской области""",1,Амбулаторный,1198,Прочие,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.2.643.5.1.13.13.12.2.91.12870,1.2.643.5.1.13.13.12.2.91.12870.0.384079,2020-03-24T10:53:54.237+0300,2022-08-23T23:03:51.469+0300,NaN,Бюро № 10,1,Амбулаторный,1198,Прочие,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.2.643.5.1.13.13.12.2.78.8657,1.2.643.5.1.13.13.12.2.78.8657.0.393657,2020-05-12T20:53:59.817+0300,2022-08-23T23:05:09.877+0300,20.06.2022,Инфекционное отделение № 10,2,Стационарный,1032,Инфекционные для взрослых,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555822,1.2.643.5.1.13.13.12.2.45.4286,1.2.643.5.1.13.13.12.2.45.4286.0.113206,2017-04-11T22:57:03.054+0300,2024-03-06T10:02:02.415+0300,NaN,Приёмное отделение,2,Стационарный,1204,Приемные отделения,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
555823,1.2.643.5.1.13.13.12.3.36.69446,1.2.643.5.1.13.13.12.3.36.69446.0.691969,2023-09-05T02:25:09.050+0300,2024-03-09T21:00:39.554+0300,NaN,Аптечный пункт №4240,7,Фармацевтический,1006,Аптеки,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
555824,1.2.643.5.1.13.13.12.3.26.68060,1.2.643.5.1.13.13.12.3.26.68060.0.667658,2023-07-07T11:37:15.428+0300,2024-03-09T20:34:12.426+0300,NaN,Аптека №3400,7,Фармацевтический,1006,Аптеки,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
555825,1.2.643.5.1.13.13.12.2.50.9555,1.2.643.5.1.13.13.12.2.50.9555.0.176758,2017-06-02T14:29:19.015+0300,2023-11-03T11:59:19.449+0300,NaN,Общебольничный персонал,6,Административно-хозяйственный (вспомогательный),1088,Организационно-методические отделы,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
df_error_example = pd.read_csv("C:/work/new_bot/new/e/08.03.2024.csv", sep=";")
df_error_example.loc[40]

session_id              921af3aa-3813-45ad-a65d-5bf156080c3c
Название субъекта РФ                               г. Москва
step_id                                                    2
step_name                                                 MO
error_code                                                 0
final_state                                Ошибка отсутсвует
mo_oid                                                   NaN
mo_name                                                  NaN
Post_name                                                NaN
slot_ts                                                  NaN
create_ts                   2024-03-08 17:45:36.517000+03:00
Тип записи                                                 3
referral_flg                                               0
Name: 40, dtype: object

In [85]:
for index, row in df_error_example.iterrows():
    if (
        pd.isnull(row["mo_oid"])
        and pd.isnull(row["mo_name"]) != True
        and row["final_state"] != "Ошибка валидации данных пациента"
        and row["final_state"] != "Ошибка во входных параметрах при записи на прием"
        and row["final_state"] != "Ошибка «РМИС не ответил»"
        and row["final_state"] != "Внутренняя ошибка РМИС при создании записи"
    ):
        print(index*100/df_error_example.shape[0], flush=True)
        mo_name_f = row["mo_name"]
        for i, r in df_dept.iterrows():
            d_name = r["depart_name"]
            kp = word_similarity(mo_name_f, d_name)
            if kp > 0.8:
                print(f"название СП в исходнике: {mo_name_f}\nназвание СП в справочнике: {d_name}\nкоэффициент похожести: {kp}\n")
            
    # break

0.03809858742160483
0.18756227653713148
0.18829494167985464
0.31138268565734717
0.3231053279409179
0.36853056678975443
0.37072856221792394
0.7495164410058027
0.8125256432799953
1.0858097415157377
1.0894730672293536
1.0902057323720766
1.121710333509173
1.1275716546509584
1.1444229529335912
1.1773928843561339
название СП в исходнике: Государственное автономное учреждение здравоохранения стоматологическая поликлиника №51 департамента здравоохранения города Москвы
название СП в справочнике: Государственное бюджетное учреждение здравоохранения города Москвы "Детская городская поликлиника № 98 Департамента здравоохранения города Москвы"
коэффициент похожести: 0.8115942028985508

1.238204091202157
1.2828966649082703
1.2968173026200105


KeyboardInterrupt: 

In [80]:
index

40

In [36]:
df_many_oid = pd.DataFrame(df_error_example.groupby(["mo_oid"])["mo_name"].nunique()).reset_index().sort_values(by="mo_name")
df_many_oid

,mo_oid,mo_name
0,1.2.643.5.1.13.13.12.2.1.11.0.345490,1
5385,1.2.643.5.1.13.13.12.2.57.5678.0.41258,1
5384,1.2.643.5.1.13.13.12.2.57.5677.0.82499,1
5383,1.2.643.5.1.13.13.12.2.57.5675.0.250667,1
5382,1.2.643.5.1.13.13.12.2.57.5672.0.109398,1
...,...,...
790,1.2.643.5.1.13.13.12.2.21.10417.0.254390,22
864,1.2.643.5.1.13.13.12.2.21.1525.0.133590,22
891,1.2.643.5.1.13.13.12.2.21.1532.0.476285,23
865,1.2.643.5.1.13.13.12.2.21.1525.0.138194,23


In [40]:
df_error_example[df_error_example["mo_oid"] == "1.2.643.5.1.13.13.12.2.21.10417.0.254478"]

,session_id,Название субъекта РФ,step_id,step_name,error_code,final_state,mo_oid,mo_name,Post_name,slot_ts,create_ts,Тип записи,referral_flg
2409,9dea8b80-fc6e-4f47-80d2-10531a16597b,Чувашская Республика,5,Slot,0,Ошибка отсутсвует,1.2.643.5.1.13.13.12.2.21.10417.0.254478,Детский участок № 17 кабинет № 126 (Здоровый п...,врач-педиатр участковый,NaN,2024-03-09 23:44:43.377 +0300,3,0
3521,e9031523-3fe9-46ac-b9a7-576127945456,Чувашская Республика,5,Slot,0,Ошибка отсутсвует,1.2.643.5.1.13.13.12.2.21.10417.0.254478,Детский участок № 4 кабинет № 114 (Здоровый пр...,врач-педиатр участковый,NaN,2024-03-09 19:31:30.017 +0300,3,0
4569,d2b70303-afbf-48bd-a8bb-f6bd553f8e0e,Чувашская Республика,4,Resource,0,Ошибка отсутсвует,1.2.643.5.1.13.13.12.2.21.10417.0.254478,Детский участок № 19 кабинет № 104 (Здоровый п...,врач-педиатр участковый,NaN,2024-03-09 20:48:40.106 +0300,3,0
8742,039a4cf4-4e81-4c77-9c7e-b2aadc4075e2,Чувашская Республика,5,Slot,0,Ошибка отсутсвует,1.2.643.5.1.13.13.12.2.21.10417.0.254478,Детский участок № 4 кабинет № 114 (Здоровый пр...,врач-педиатр участковый,NaN,2024-03-09 20:07:52.470 +0300,3,0
8848,857178c0-fc89-46b6-8dd9-b1f1c241fa46,Чувашская Республика,5,Slot,0,Ошибка отсутсвует,1.2.643.5.1.13.13.12.2.21.10417.0.254478,Детский участок № 9 кабинет № 118 (Здоровый пр...,врач-педиатр участковый,NaN,2024-03-09 07:35:07.728 +0300,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159869,35292543-ecd5-4113-9e1a-3cdc5a1eb17d,Чувашская Республика,3,ServiceOrSpecs,0,Ошибка отсутсвует,1.2.643.5.1.13.13.12.2.21.10417.0.254478,Детские узкие специалисты. Поликлиника № 3. - ...,NaN,NaN,2024-03-09 18:29:54.665 +0300,3,0
162163,d8d2b890-529f-4bf1-9e7e-81a609ce8022,Чувашская Республика,5,Slot,0,Ошибка отсутсвует,1.2.643.5.1.13.13.12.2.21.10417.0.254478,Детские узкие специалисты. Поликлиника № 3. - ...,врач-травматолог-ортопед,NaN,2024-03-09 11:53:09.320 +0300,3,0
169214,20dd1d9d-e1b5-46bc-94fa-9cc34101d729,Чувашская Республика,5,Slot,0,Ошибка отсутсвует,1.2.643.5.1.13.13.12.2.21.10417.0.254478,Детские узкие специалисты. Поликлиника № 3. - ...,врач-оториноларинголог,NaN,2024-03-09 07:34:24.276 +0300,3,0
172254,8817c85d-014c-48eb-ae5c-f349056eb877,Чувашская Республика,5,Slot,0,Ошибка отсутсвует,1.2.643.5.1.13.13.12.2.21.10417.0.254478,Детские узкие специалисты. Поликлиника № 3. - ...,врач-оториноларинголог,NaN,2024-03-09 07:49:57.534 +0300,3,0


***

**ПРИЧЁСЫВАНИЕ ГОСУДАРСТВЕННОГО КАТАЛОГА ГЕОГРАФИЧЕСКИХ НАЗВАНИЙ**

In [141]:
file_path = "C:/work/scripts/справочники/мутный_справочник/Мутный_справочник/Алтайский край_202нп.xlsx"
wb = xl.load_workbook(file_path)
sheets_list = wb.sheetnames
df_result = pd.DataFrame()
for sheet_name in sheets_list:
    df_tmp = pd.read_excel(file_path, sheet_name=sheet_name, index_col=None, dtype="str")
    if sheet_name == "Sheet1":
        region_name = df_tmp.loc[2, "Unnamed: 0"]
        df_tmp = df_tmp.drop(
            index=[x for x in range(0, df_tmp[df_tmp["Unnamed: 0"] == "1"].index[0] + 1)]
        )
        df_tmp = df_tmp.drop("Unnamed: 6", axis=1)
        df_tmp = df_tmp.reset_index(drop=True)
        df_tmp["Unnamed: 2"] = df_tmp["Unnamed: 2"].str.replace("\n", "")
        df_tmp["Unnamed: 0"] = df_tmp["Unnamed: 0"].str.replace("\n", "")
        df_tmp["Unnamed: 0"] = df_tmp["Unnamed: 0"].apply(
            lambda x: re.sub("количество записей - \d*", "", str(x))
        )
        df_tmp["Unnamed: 0"] = df_tmp["Unnamed: 0"].apply(
            lambda x: re.sub("\d*", "", str(x))
        )
        df_tmp["Unnamed: 0"] = df_tmp["Unnamed: 0"].apply(lambda x: x.replace("nan", ""))
        df_tmp["Unnamed: 0"] = df_tmp["Unnamed: 0"].fillna("")
        for index, row in df_tmp.iterrows():
            if row["Unnamed: 0"] == "" and index != 0:
                df_tmp.at[index, "Unnamed: 0"] = df_tmp.loc[index - 1, "Unnamed: 0"]
        df_tmp.dropna(subset=["Unnamed: 5"], inplace=True)
        df_tmp.insert(0, "region_name", region_name)
        df_tmp = df_tmp.rename(
            columns={
                df_tmp.columns[1]: "municipality",
                df_tmp.columns[2]: "settelment",
                df_tmp.columns[3]: "type",
                df_tmp.columns[4]: "latitude",
                df_tmp.columns[5]: "longitude",
                df_tmp.columns[6]: "map_number",
            }
        )
    else:
        df_tmp[1] = df_tmp[1].apply(lambda x: re.sub("количество записей - \d*", "", str(x)))
        df_tmp[1] = df_tmp[1].apply(lambda x: re.sub("\d*", "", str(x)))
        for index, row in df_tmp.iterrows():
            if row[1] == "" and index !=0:
                df_tmp.at[index, 1] = df_tmp.loc[index-1, 1]
        df_tmp.dropna(subset=["Unnamed: 5"], inplace=True)
        df_tmp.insert(0, "region_name", "")
        df_tmp = df_tmp.rename(
            columns={
                df_tmp.columns[1]: "municipality",
                df_tmp.columns[2]: "settelment",
                df_tmp.columns[3]: "type",
                df_tmp.columns[4]: "latitude",
                df_tmp.columns[5]: "longitude",
                df_tmp.columns[6]: "map_number",
            }
        )
    df_result = pd.concat([df_result, df_tmp])
df_result = df_result.reset_index(drop=True)
df_result["municipality"] = df_result["municipality"].str.replace("nan", "")
for index, row in df_result.iterrows():
        if row["region_name"] == "" and index !=0:
            df_result.at[index, "region_name"] = df_result.loc[index-1, "region_name"]
        if row["municipality"] == "" and index !=0:
            df_result.at[index, "municipality"] = df_result.loc[index-1, "municipality"]
df_result

,region_name,municipality,settelment,type,latitude,longitude,map_number,Unnamed: 6
0,Алтайский край,"Населённые пункты, не относящиеся к районам",Алейск,город краев.знач.,52°29.1',82°48.1',N-44-130,NaN
1,Алтайский край,"Населённые пункты, не относящиеся к районам",Барнаул,город краев.знач.,53°21.7',83°42.9',N-44-096,NaN
2,Алтайский край,"Населённые пункты, не относящиеся к районам",Белокуриха,город краев.знач.,51°59.8',84°58.7',M-45-002,NaN
3,Алтайский край,"Населённые пункты, не относящиеся к районам",Бийск,город краев.знач.,52°32.3',85°12.4',N-45-123,NaN
4,Алтайский край,"Населённые пункты, не относящиеся к районам",Заринск,город краев.знач.,53°42.3',84°57.2',N-45-074,NaN
...,...,...,...,...,...,...,...,...
1604,Алтайский край,город краевого значения Славгород,Райгород,село,52°51.7',78°24.4',N-44-109,NaN
1605,Алтайский край,город краевого значения Славгород,Селекционное,село,52°58.7',78°36',N-44-110,NaN
1606,Алтайский край,город краевого значения Славгород,Семеновка,село,53°05.3',78°58.8',N-44-098,NaN
1607,Алтайский край,город краевого значения Славгород,Славгородское,село,53°00.8',78°38.4',N-44-098,NaN


In [142]:
# load the workbook 
# wb = xl.load_workbook("C:/work/scripts/справочники/мутный_справочник/Мутный_справочник/Амурская область_202нп.xlsx") 
  
# print the list that stores the sheetnames 
# print(wb.sheetnames) 
  
# we can also get the name of the sheet which is active by using the active property 
# print("Active sheet: ", wb.active) 
  
# By default the active sheet is the 1st one and can be changed 
# wb._active_sheet_index = 4
# sheet = wb.active 
# print("Active sheet: ", wb.active) 

In [39]:
source_path = "C:/work/gkgn/source/"
result_path = "C:/work/gkgn/result/"
list_files = os.listdir(source_path)
for file_name in list_files:
    try:
        file_path = source_path + file_name
        wb = xl.load_workbook(file_path)
        sheets_list = wb.sheetnames
        df_result = pd.DataFrame()
        for sheet_name in sheets_list:
            df_tmp = pd.read_excel(file_path, sheet_name=sheet_name, index_col=None, dtype="str")
            df_0 = df_tmp.copy()
            if sheet_name == "Sheet1":
                region_name = df_tmp.loc[2, "Unnamed: 0"]
                df_tmp = df_tmp.drop(
                    index=[x for x in range(0, df_tmp[df_tmp["Unnamed: 0"] == "1"].index[0] + 1)]
                )
                df_tmp = df_tmp.drop("Unnamed: 6", axis=1)
                df_tmp = df_tmp.reset_index(drop=True)
                df_tmp["Unnamed: 2"] = df_tmp["Unnamed: 2"].str.replace("\n", "")
                df_tmp["Unnamed: 0"] = df_tmp["Unnamed: 0"].str.replace("\n", "")
                df_tmp["Unnamed: 0"] = df_tmp["Unnamed: 0"].apply(
                    lambda x: re.sub("количество записей - \d*", "", str(x))
                )
                df_tmp["Unnamed: 0"] = df_tmp["Unnamed: 0"].apply(
                    lambda x: re.sub("\d*", "", str(x))
                )
                df_tmp["Unnamed: 0"] = df_tmp["Unnamed: 0"].apply(lambda x: x.replace("nan", ""))
                df_tmp["Unnamed: 0"] = df_tmp["Unnamed: 0"].fillna("")
                for index, row in df_tmp.iterrows():
                    if row["Unnamed: 0"] == "" and index != 0:
                        df_tmp.at[index, "Unnamed: 0"] = df_tmp.loc[index - 1, "Unnamed: 0"]
                df_tmp.dropna(subset=["Unnamed: 5"], inplace=True)
                df_tmp.insert(0, "region_name", region_name)
                df_tmp = df_tmp.rename(
                    columns={
                        df_tmp.columns[1]: "municipality",
                        df_tmp.columns[2]: "settelment",
                        df_tmp.columns[3]: "type",
                        df_tmp.columns[4]: "latitude",
                        df_tmp.columns[5]: "longitude",
                        df_tmp.columns[6]: "map_number",
                    }
                )
            else:
                try:
                    df_tmp[1] = df_tmp[1].apply(lambda x: re.sub("количество записей - \d*", "", str(x)))
                    df_tmp[1] = df_tmp[1].apply(lambda x: re.sub("\d*", "", str(x)))
                    for index, row in df_tmp.iterrows():
                        if row[1] == "" and index !=0:
                            df_tmp.at[index, 1] = df_tmp.loc[index-1, 1]
                    # df_tmp.dropna(subset=[df_tmp.columns[-1]], inplace=True)
                    df_tmp.insert(0, "region_name", "")
                    df_tmp = df_tmp.rename(
                        columns={
                            df_tmp.columns[1]: "municipality",
                            df_tmp.columns[2]: "settelment",
                            df_tmp.columns[3]: "type",
                            df_tmp.columns[4]: "latitude",
                            df_tmp.columns[5]: "longitude",
                            df_tmp.columns[6]: "map_number",
                        }
                    )
                except Exception as e_0:
                    print(f"файл {file_path}, лист {sheet_name} не обработан, ибо {e_0}")
            df_result = pd.concat([df_result, df_tmp])
        df_result = df_result.reset_index(drop=True)
        df_result["municipality"] = df_result["municipality"].str.replace("nan", "")
        for index, row in df_result.iterrows():
                if row["region_name"] == "" and index !=0:
                    df_result.at[index, "region_name"] = df_result.loc[index-1, "region_name"]
                    print()
                if row["municipality"] == "" and index !=0:
                    df_result.at[index, "municipality"] = df_result.loc[index-1, "municipality"]
        result_path_file = result_path + "_".join(region_name.split(" ")) + ".csv"
        df_result.to_csv(result_path_file, sep=";", encoding="utf-8-sig", index=False)
        print(f"сохранено {result_path_file}")
        # break
    except Exception as e:
        print(f"не получилось {file_path} ибо {e}")




























































































































































сохранено C:/work/gkgn/result/Мурманская_область.csv


In [30]:
df_0

,1,2,3,4,5,Unnamed: 5
0,зато Александровск,NaN,NaN,NaN,NaN,NaN
1,количество записей - 8,NaN,NaN,NaN,NaN,NaN
2,0054225,Белокаменка,село,69°04',33°10',R-36-103
3,0010665,Гаджиево,город,69°14.9',33°19.2',R-36-103
4,0054292,Кувшинская Салма,нп,69°18.3',33°25.4',R-36-103
5,0054250,Оленья Губа,нп,69°12.8',33°22.2',R-36-103
6,0010663,Полярный,город,69°11.9',33°26.9',R-36-103
7,0054230,Ретинское,нп,69°06.8',33°22.4',R-36-103
8,0054120,Сайда-Губа,нп,69°14.2',33°11.5',R-36-103
9,0036999,Снежногорск,город,69°11.5',33°14.1',R-36-103


In [20]:
# разделение по количеству столбцов
result_path = "C:/work/gkgn/result/"
_7_path = "C:/work/gkgn/col_7/"
_8_path = "C:/work/gkgn/col_8/"
_9_path = "C:/work/gkgn/col_8+/"
file_list = os.listdir(result_path)
for file_name in file_list:
    df_tmp = pd.read_csv(result_path + file_name, sep=";")
    if df_tmp.shape[1] == 7:
        df_tmp.to_csv(_7_path + file_name, sep=";", encoding="utf-8-sig", index=False)
        print(f"в файле {file_name} 7 полей, сохранено в {_7_path + file_name}")
    elif df_tmp.shape[1] == 8:
        df_tmp.to_csv(_8_path + file_name, sep=";", encoding="utf-8-sig", index=False)
        print(f"в файле {file_name} 8 полей, сохранено в {_8_path + file_name}")
    else:
        df_tmp.to_csv(_9_path + file_name, sep=";", encoding="utf-8-sig", index=False)
        print(f"в файле {file_name} {df_tmp.shape[1]} полей, сохранено в {_9_path + file_name}")

в файле Алтайский_край.csv 7 полей, сохранено в C:/work/gkgn/col_7/Алтайский_край.csv
в файле Амурская_область.csv 7 полей, сохранено в C:/work/gkgn/col_7/Амурская_область.csv
в файле Архангельская_область.csv 7 полей, сохранено в C:/work/gkgn/col_7/Архангельская_область.csv
в файле Астраханская_область.csv 7 полей, сохранено в C:/work/gkgn/col_7/Астраханская_область.csv
в файле Белгородская_область.csv 7 полей, сохранено в C:/work/gkgn/col_7/Белгородская_область.csv
в файле Брянская_область.csv 8 полей, сохранено в C:/work/gkgn/col_8/Брянская_область.csv
в файле Владимирская_область.csv 8 полей, сохранено в C:/work/gkgn/col_8/Владимирская_область.csv
в файле Волгоградская_область.csv 7 полей, сохранено в C:/work/gkgn/col_7/Волгоградская_область.csv
в файле Вологодская_область.csv 8 полей, сохранено в C:/work/gkgn/col_8/Вологодская_область.csv
в файле Воронежская_область.csv 9 полей, сохранено в C:/work/gkgn/col_8+/Воронежская_область.csv
в файле город_федерального_значения_Москва.csv 

In [76]:
# переработка 8-столбцовых в нормальные
_7_path = "C:/work/gkgn/col_7/"
_8_path = "C:/work/gkgn/col_8/"
file_list_8 = os.listdir(_8_path)
for file_name in file_list_8:
    try:
        df_tmp = pd.read_csv(_8_path + file_name, sep=";")
        df_tmp[df_tmp.columns[-1]] = df_tmp[df_tmp.columns[-1]].fillna("del")
        col_8_name = df_tmp.columns[-1]
        df_7 = df_tmp[df_tmp[col_8_name] == "del"]
        df_7 = df_7.drop(col_8_name, axis=1)
        df_8 = df_tmp[df_tmp[col_8_name] != "del"]
        df_8["real_settelment"] = df_8.apply(
            lambda row: (
                row["settelment"]
                if pd.isnull(row["type"])
                else str(row["settelment"]) + " " + str(row["type"])
            ),
            axis=1,
        )
        df_8 = df_8.drop("settelment", axis=1)
        df_8 = df_8[
            [
                "region_name",
                "municipality",
                "real_settelment",
                "latitude",
                "longitude",
                "map_number",
                "Unnamed: 6",
            ]
        ]
        df_8 = df_8.rename(
            columns={
                "real_settelment": "settelment",
                "latitude": "type",
                "longitude": "latitude",
                "map_number": "longitude",
                "Unnamed: 6": "map_number",
            }
        )
        df_tmp = pd.concat([df_7, df_8])
        df_tmp = df_tmp.reset_index(drop=True)
        df_tmp.to_csv(_7_path + "new/" + file_name, sep=";", encoding="utf-8-sig", index=False)
        print(f"сделано {file_name}")
    except Exception as e:
        print(f"не получилось {file_name} ибо {e}")
    # break

сделано Брянская_область.csv
сделано Владимирская_область.csv
сделано Вологодская_область.csv
сделано Воронежская_область.csv
сделано Донецкая_Народная_Республика_субъект_Российской_Федерации.csv
сделано Забайкальский_край.csv
сделано Запорожская_область_субъект_Российской_Федерации.csv
сделано Ивановская_область.csv
сделано Иркутская_область.csv
сделано Кабардино-Балкарская_Республика.csv
сделано Калужская_область.csv
сделано Карачаево-Черкесская_Республика.csv
сделано Кемеровская_область_-_Кузбасс.csv
сделано Кировская_область.csv
сделано Костромская_область.csv
сделано Краснодарский_край.csv
сделано Красноярский_край.csv
сделано Курганская_область.csv
сделано Курская_область.csv
сделано Ленинградская_область.csv
сделано Липецкая_область.csv
сделано Московская_область.csv
сделано Мурманская_область.csv
сделано Нижегородская_область.csv
сделано Новгородская_область.csv
сделано Новосибирская_область.csv
сделано Омская_область.csv
сделано Орловская_область.csv
сделано Пензенская_область

In [82]:
# объединение Руси (Иван Калита, ага...)
_7_path = "C:/work/gkgn/col_7/"
file_list_7 = os.listdir(_7_path)
df_russia = pd.DataFrame()
for file_name in file_list_7:
    df_tmp = pd.read_csv(_7_path + file_name, sep=";")
    try:
        df_russia = pd.concat([df_russia, df_tmp])
        print(f" Я присоединил к Руси {file_name}")
    except Exception as e:
        print (f"не присоединилось {file_name}, ибо {e}")
df_russia = df_russia.reset_index(drop=True)
df_russia

 Я присоединил к Руси Алтайский_край.csv
 Я присоединил к Руси Амурская_область.csv
 Я присоединил к Руси Архангельская_область.csv
 Я присоединил к Руси Астраханская_область.csv
 Я присоединил к Руси Белгородская_область.csv
 Я присоединил к Руси Брянская_область.csv
 Я присоединил к Руси Владимирская_область.csv
 Я присоединил к Руси Волгоградская_область.csv
 Я присоединил к Руси Вологодская_область.csv
 Я присоединил к Руси Воронежская_область.csv
 Я присоединил к Руси город_федерального_значения_Москва.csv
 Я присоединил к Руси город_федерального_значения_Санкт-Петербург.csv
 Я присоединил к Руси город_федерального_значения_Севастополь.csv
 Я присоединил к Руси Донецкая_Народная_Республика_субъект_Российской_Федерации.csv
 Я присоединил к Руси Еврейская_автономная_область.csv
 Я присоединил к Руси Забайкальский_край.csv
 Я присоединил к Руси Запорожская_область_субъект_Российской_Федерации.csv
 Я присоединил к Руси Ивановская_область.csv
 Я присоединил к Руси Иркутская_область.csv

,region_name,municipality,settelment,type,latitude,longitude,map_number
0,Алтайский край,"Населённые пункты, не относящиеся к районам",Алейск,город краев.знач.,52°29.1',82°48.1',N-44-130
1,Алтайский край,"Населённые пункты, не относящиеся к районам",Барнаул,город краев.знач.,53°21.7',83°42.9',N-44-096
2,Алтайский край,"Населённые пункты, не относящиеся к районам",Белокуриха,город краев.знач.,51°59.8',84°58.7',M-45-002
3,Алтайский край,"Населённые пункты, не относящиеся к районам",Бийск,город краев.знач.,52°32.3',85°12.4',N-45-123
4,Алтайский край,"Населённые пункты, не относящиеся к районам",Заринск,город краев.знач.,53°42.3',84°57.2',N-45-074
...,...,...,...,...,...,...,...
153438,Ярославская область,Ярославский район,Красный Бор,деревня,57°38.8',40°01',O-37-093
153439,Ярославская область,Ярославский район,Красный Волгарь,посёлок,57°47.1',39°41.6',O-37-080
153440,Ярославская область,Ярославский район,Красный Холм,посёлок,57°47.2',39°42.2',O-37-080
153441,Ярославская область,Ярославский район,Крюково,деревня,57°32.2',39°31.1',O-37-092


In [83]:
df_russia

,region_name,municipality,settelment,type,latitude,longitude,map_number
0,Алтайский край,"Населённые пункты, не относящиеся к районам",Алейск,город краев.знач.,52°29.1',82°48.1',N-44-130
1,Алтайский край,"Населённые пункты, не относящиеся к районам",Барнаул,город краев.знач.,53°21.7',83°42.9',N-44-096
2,Алтайский край,"Населённые пункты, не относящиеся к районам",Белокуриха,город краев.знач.,51°59.8',84°58.7',M-45-002
3,Алтайский край,"Населённые пункты, не относящиеся к районам",Бийск,город краев.знач.,52°32.3',85°12.4',N-45-123
4,Алтайский край,"Населённые пункты, не относящиеся к районам",Заринск,город краев.знач.,53°42.3',84°57.2',N-45-074
...,...,...,...,...,...,...,...
153438,Ярославская область,Ярославский район,Красный Бор,деревня,57°38.8',40°01',O-37-093
153439,Ярославская область,Ярославский район,Красный Волгарь,посёлок,57°47.1',39°41.6',O-37-080
153440,Ярославская область,Ярославский район,Красный Холм,посёлок,57°47.2',39°42.2',O-37-080
153441,Ярославская область,Ярославский район,Крюково,деревня,57°32.2',39°31.1',O-37-092


In [95]:
# после ручной обработки
rus_path = "C:/work/gkgn/df_russia_result.xlsx"
df_rus = pd.read_excel(rus_path, sheet_name="to_python", index_col=None, dtype="str")
df_rus["latitude_g"] = df_rus["latitude"].apply(lambda x: int(x.split("°")[0]) + float(x.split("°")[1].split("'")[0])/60)
df_rus["longitude_g"] = df_rus["longitude"].apply(lambda x: int(x.split("°")[0]) + float(x.split("°")[1].split("'")[0])/60)
df_rus.to_csv("C:/work/gkgn/df_rus_0.csv", sep=";", encoding="utf-8-sig", index=False)

**ПРИВЯЗЫВАНИЕ ГОРОДСКОГО НАСЕЛЕНИЯ ИЗ РОССТАТА**

In [22]:
rus_path = "C:\work\scripts\справочники\ГКГН.xlsx"
rosstat_path = "C:\work\scripts\справочники\c&t.csv"
df_rus = pd.read_excel(rus_path, sheet_name="df_rus", index_col=None, dtype="str")
df_stat = pd.read_csv(rosstat_path, sep=";")

In [23]:
df_rus["reg_merge"] = df_rus["region_name"].str.lower().str.replace(r'\W', '', regex=True).str.replace("ё", "е")
df_rus["slm_merge"] = df_rus["settelment"].str.lower().str.replace(r'\W', '', regex=True).str.replace("ё", "е")
df_rus

,region_name,municipality,settelment,type,latitude,longitude,latitude_g,longitude_g,reg_merge,slm_merge
0,Алтайский край,"Населённые пункты, не относящиеся к районам",Алейск,город краев.знач.,52°29.1',82°48.1',52.485,82.8016666666666,алтайскийкрай,алейск
1,Алтайский край,"Населённые пункты, не относящиеся к районам",Барнаул,город краев.знач.,53°21.7',83°42.9',53.3616666666666,83.715,алтайскийкрай,барнаул
2,Алтайский край,"Населённые пункты, не относящиеся к районам",Белокуриха,город краев.знач.,51°59.8',84°58.7',51.9966666666666,84.9783333333333,алтайскийкрай,белокуриха
3,Алтайский край,"Населённые пункты, не относящиеся к районам",Бийск,город краев.знач.,52°32.3',85°12.4',52.5383333333333,85.2066666666666,алтайскийкрай,бийск
4,Алтайский край,"Населённые пункты, не относящиеся к районам",Заринск,город краев.знач.,53°42.3',84°57.2',53.705,84.9533333333333,алтайскийкрай,заринск
...,...,...,...,...,...,...,...,...,...,...
153372,Ярославская область,Ярославский район,Красный Бор,деревня,57°38.8',40°01',57.6466666666666,40.0166666666666,ярославскаяобласть,красныйбор
153373,Ярославская область,Ярославский район,Красный Волгарь,посёлок,57°47.1',39°41.6',57.785,39.6933333333333,ярославскаяобласть,красныйволгарь
153374,Ярославская область,Ярославский район,Красный Холм,посёлок,57°47.2',39°42.2',57.7866666666666,39.7033333333333,ярославскаяобласть,красныйхолм
153375,Ярославская область,Ярославский район,Крюково,деревня,57°32.2',39°31.1',57.5366666666666,39.5183333333333,ярославскаяобласть,крюково


In [24]:
df_stat["reg_merge"] = df_stat["region_name"].str.lower().str.replace(r'\W', '', regex=True).str.replace("ё", "е")
df_stat["slm_merge"] = df_stat["area"].str.lower().str.replace(r'\W', '', regex=True).str.replace("ё", "е")
df_stat

,region_name,pref,area,population,reg_merge,slm_merge
0,алтайский край,г,барнаул,623057,алтайскийкрай,барнаул
1,алтайский край,г,бийск,181678,алтайскийкрай,бийск
2,алтайский край,г,рубцовск,124687,алтайскийкрай,рубцовск
3,алтайский край,г,новоалтайск,73122,алтайскийкрай,новоалтайск
4,алтайский край,г,заринск,40626,алтайскийкрай,заринск
...,...,...,...,...,...,...
2280,ярославская область,пгт,бурмакино,2554,ярославскаяобласть,бурмакино
2281,ярославская область,пгт,лесная поляна,2460,ярославскаяобласть,леснаяполяна
2282,ярославская область,пгт,поречье-рыбное,1405,ярославскаяобласть,поречьерыбное
2283,ярославская область,пгт,красный профинтерн,1076,ярославскаяобласть,красныйпрофинтерн


In [25]:
df_rus_stat = df_rus.merge(df_stat, on=["reg_merge", "slm_merge"], how="outer")
df_rus_stat

,region_name_x,municipality,settelment,type,latitude,longitude,latitude_g,longitude_g,reg_merge,slm_merge,region_name_y,pref,area,population
0,Алтайский край,"Населённые пункты, не относящиеся к районам",Алейск,город краев.знач.,52°29.1',82°48.1',52.485,82.8016666666666,алтайскийкрай,алейск,алтайский край,г,алейск,24804.0
1,Алтайский край,"Населённые пункты, не относящиеся к районам",Барнаул,город краев.знач.,53°21.7',83°42.9',53.3616666666666,83.715,алтайскийкрай,барнаул,алтайский край,г,барнаул,623057.0
2,Алтайский край,"Населённые пункты, не относящиеся к районам",Белокуриха,город краев.знач.,51°59.8',84°58.7',51.9966666666666,84.9783333333333,алтайскийкрай,белокуриха,алтайский край,г,белокуриха,14791.0
3,Алтайский край,"Населённые пункты, не относящиеся к районам",Бийск,город краев.знач.,52°32.3',85°12.4',52.5383333333333,85.2066666666666,алтайскийкрай,бийск,алтайский край,г,бийск,181678.0
4,Алтайский край,"Населённые пункты, не относящиеся к районам",Заринск,город краев.знач.,53°42.3',84°57.2',53.705,84.9533333333333,алтайскийкрай,заринск,алтайский край,г,заринск,40626.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,хантымансийскийавтономныйокругюгра,новоаганск,ханты-мансийский автономный округ - югра,пгт,новоаганск,9034.0
153419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,челябинскаяобласть,сатка,челябинская область,г,сатка,42062.0
153420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,челябинскаяобласть,бакал,челябинская область,г,бакал,16103.0
153421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,челябинскаяобласть,бердяуш,челябинская область,пгт,бердяуш,3860.0


In [26]:
df_rus_stat.to_csv("C:/work/gkgn/df_rus_stat.csv", sep=";", encoding="utf-8-sig", index=False)

In [27]:
anton_path = "C:/work/scripts/справочники/big_population.csv"
df_anton = pd.read_csv(anton_path, sep=";")
df_anton

,id,region,municipality,settlement,type,population,children,latitude_dms,longitude_dms,latitude_dd,longitude_dd,oktmo,dadata,rosstat
0,0,Орловская область,Болховский,Колонтаева,д,0,0,53.22.07,035.54.36,53.368611,35.910000,5.460442e+10,0,1
1,1,Республика Крым,Алушта,Пушкино,с,273,0,44.35.45,034.20.27,44.595833,34.340833,3.570300e+10,0,1
2,2,Липецкая область,Лев-Толстовский район,Барятино,с,7,1,53.15.46,039.30.14,53.262778,39.503889,4.263641e+10,0,1
3,3,Тверская область,Селижаровский район,Хилово,д,2,0,56.54.20,033.25.09,56.905556,33.419167,2.865043e+10,0,1
4,4,Томская область,Парабельский район,Басмасово,д,6,0,58.38.12,082.02.40,58.636667,82.044444,6.964444e+10,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155916,155916,Ульяновская область,Вешкаймский район,Ребровка,д,0,0,53.59.12,046.48.35,53.986667,46.809722,7.360744e+10,0,1
155917,155917,Псковская область,Палкинский район,Шадрица,д,1,0,57.29.03,028.08.48,57.484167,28.146667,5.863742e+10,0,1
155918,155918,Псковская область,Себежский район,Овчинниково,д,0,0,56.19.48,028.26.17,56.330000,28.438056,5.865410e+10,0,1
155919,155919,Тульская область,рабочий пос. Новогуровский,Новогуровский,рп,3398,305,54.28.03,037.20.19,54.467500,37.338611,7.070200e+10,1,0


**ВСЁ ВЗАД!**

In [3]:
source_path = "C:/work/gkgn/source/"
result_path = "C:/work/gkgn/new_result/"
list_files = os.listdir(source_path)
cols = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for file_name in list_files:
    try:
        df_tmp = pd.DataFrame()
        try:
            file_path = source_path + file_name
            wb = xl.load_workbook(file_path)
            sheets_list = wb.sheetnames
            for sheet_name in sheets_list:
                df_t = pd.read_excel(file_path, sheet_name=sheet_name, index_col=None, dtype="str", header=None, names=cols)
                print(f"лист {sheet_name}")
                df_tmp = pd.concat([df_tmp, df_t])
        except Exception as e:
            print(f"не получился лист {sheet_name} из {file_name} ибо {e}")
        print(f"последний лист {sheet_name}")
        df_tmp = df_tmp.reset_index(drop=True)
        df_tmp = df_tmp.fillna("nan")
        df_test = df_tmp.copy()
        for col_names in df_tmp.columns:
            df_tmp[col_names] = df_tmp[col_names].str.replace("\n", " ").str.replace("-", " ")
            df_tmp[col_names] = df_tmp[col_names].str.replace("несогласованные названия географических объектов", "")
            df_tmp[col_names] = df_tmp[col_names].str.replace("расхождение в форме написания", "")
            df_tmp[col_names] = df_tmp[col_names].apply(lambda x: re.sub("Общее количество записей\s+\d*", "", str(x)))
            df_tmp[col_names] = df_tmp[col_names].apply(lambda x: re.sub("количество записей\s+\d*", "", str(x)))
        df_test_1 = df_tmp.copy()
        region_name = df_tmp.loc[3, 1]
        # .iloc[0]
        print(f"определён регион {region_name}")
        del_index = df_tmp[df_tmp[1] == "1"].index[0]
        df_tmp = df_tmp.drop(index=[x for x in range(del_index+1)])
        print(f"удалены строки по {del_index}")
        region_list = []
        area_name = ""
        for index, row in df_tmp.iterrows():
            value_1 = row[1]
            value_2 = row[2]
            value_3 = row[3]
            if (pd.isna(value_1) or value_1 == "" or value_1 == "nan") and (pd.isna(value_2) or value_2 == "" or value_2 == "nan"):
                df_tmp.at[index, 1] = value_3
                df_tmp.at[index, 3] = "nan"
                print(f"{index}: замена {value_1} на {value_3} (1-3)")
            elif pd.isna(value_1) or value_1 == "" or value_1 == "nan":
                df_tmp.at[index, 1] = value_2
                df_tmp.at[index, 2] = "nan"
                print(f"{index}: замена {value_1} на {value_1} (1-2)")
        df_tmp = df_tmp.reset_index(drop=True)
        df_test_1 = df_tmp.copy()
        for index, row in df_tmp.iterrows():
            if re.fullmatch(r"[а-яА-ЯёЁ,\s]+[количество записей -]*", row[1].strip()):
                area_name = row[1]
                print(f"\n{index}: определён район {area_name}\n")
            elif re.fullmatch(r"\d{4,8}", row[1]):
                n=0
                for col_name in df_tmp.columns:
                    if re.search(r"\d+°\d*.*\d*'", row[col_name]) and n == 0:
                        lat = row[col_name]
                        cord_col = col_name
                        row_index = index
                        n+=1
                    if re.search(r"\d+°\d*.*\d*'", row[col_name]) and n == 1:
                        lon = row[col_name]
                # print(f"координаты на строке {index}")
                s_type = df_tmp.loc[index, cord_col-1]
                # print("префикс")
                if index == index:
                    settelment = row[2]
                    for col_n in range(3, cord_col-1):
                        row[col_n]
                        settelment = settelment + " " + row[col_n]
                print(f"{index}: {settelment}, {area_name}")
                region_list.append(file_name.split("_")[0])
                region_list.append(region_name)
                region_list.append(area_name)
                region_list.append(s_type)
                region_list.append(settelment)
                region_list.append(lat)
                region_list.append(lon)
            else:
                # df_bad_rows.loc[len(df_bad_rows)] = row
                print(f"{index}: плохая строка")
        reg_list = [region_list[i : i + 7] for i in range(0, len(region_list), 7)]
        df_region = pd.DataFrame(
            reg_list,
            columns=[
                "file_name",
                "region",
                "municipality",
                "type",
                "settelment",
                "latitude",
                "longitude"
            ],
        )
        for column_name in df_region.columns:
            df_region[column_name] = df_region[column_name].str.replace(" nan", "")
        df_region.drop_duplicates()
        # df_region = df_region.sort_values(by=["municipality", "settelment"])
        df_region.to_csv(result_path + "_".join(file_name.split("_")[0].split(" ")) + ".csv", sep=";", encoding="utf-8-sig", index=False)
        print(f"получилось {file_name} как {region_name}")
    except Exception as e:
        print(f"не получилось {file_name} ибо {e}")
    q = input(f"Сделано {region_name} СЛЕДУЮЩИЙ РЕГИОН? y/n")
    if q == "n":
        break
    else:
        clear_output()

In [11]:
ikalita_path = "C:/work/gkgn/ivan_kalita.xlsx"
df_rus = pd.read_excel(ikalita_path, sheet_name="new_result", index_col=None, dtype="str")
df_rus["latitude_g"] = df_rus["latitude"].apply(lambda x: int(x.split("°")[0]) + float(x.split("°")[1].split("'")[0])/60)
df_rus["longitude_g"] = df_rus["longitude"].apply(lambda x: int(x.split("°")[0]) + float(x.split("°")[1].split("'")[0])/60)
df_rus.to_csv("C:/work/gkgn/rus.csv", sep=";", encoding="utf-8-sig", index=False)

In [3]:
rus_path = "C:/work/gkgn/rus.xlsx"
df_rus = pd.read_excel(rus_path, sheet_name="rus", index_col=None, dtype="str")

In [9]:
df_rus_pop = df_rus.copy()
df_rus_pop["reg_merge"] = df_rus_pop["region"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_rus_pop["area_merge"] = df_rus_pop["settelment"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')

In [11]:
df_rus_pop

,region,municipality,type,settelment,latitude,longitude,latitude_g,longitude_g,reg_merge,area_merge
0,Алтайский край,Алейский район,посёлок,Алейский,52°28.2',82°43.7',52.47,82.7283333333333,алтайскийкрай,алейский
1,Алтайский край,Алейский район,посёлок,Александровский,52°23.2',82°55.4',52.3866666666666,82.9233333333333,алтайскийкрай,александровский
2,Алтайский край,Алейский район,село,Безголосово,52°31',82°58',52.5166666666666,82.9666666666666,алтайскийкрай,безголосово
3,Алтайский край,Алейский район,посёлок,Березовский,52°34.1',82°18.5',52.5683333333333,82.3083333333333,алтайскийкрай,березовский
4,Алтайский край,Алейский район,село,Большепанюшево,52°31',82°52.2',52.5166666666666,82.87,алтайскийкрай,большепанюшево
...,...,...,...,...,...,...,...,...,...,...
156947,Ярославская область,Ярославский район,деревня,Ямино,57°45',39°41.7',57.75,39.695,ярославскаяобласть,ямино
156948,Ярославская область,Ярославский район,деревня,Ямищи,57°31.8',39°53.9',57.53,39.8983333333333,ярославскаяобласть,ямищи
156949,Ярославская область,Ярославский район,посёлок,Ярославка,57°44.5',39°50.6',57.7416666666666,39.8433333333333,ярославскаяобласть,ярославка
156950,Ярославская область,Ярославский район,деревня,Ярцево,57°32.2',40°00.4',57.5366666666666,40.0066666666666,ярославскаяобласть,ярцево


In [6]:
# численность городского населения Росстат
russtat_path = "C:\work\scripts\справочники\c&t.csv"
df_russtat = pd.read_csv(russtat_path, sep=";")
# антонов огонь
df_big_pop = pd.read_csv("C:/work/scripts/справочники/big_population.csv", sep=";", index_col=None)
# df_big_pop["region"] = df_big_pop["region"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
# df_big_pop["municipality"] = df_big_pop["municipality"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
# df_big_pop["settlement"] = df_big_pop["settlement"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
# df_big_pop["type"] = df_big_pop["type"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
# df_big_pop = df_big_pop.drop("id", axis=1)

In [15]:
df_russtat["reg_merge"] = df_russtat["region_name"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_russtat["area_merge"] = df_russtat["area"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_russtat

,region_name,pref,area,population,reg_merge,area_merge
0,алтайский край,г,барнаул,623057,алтайскийкрай,барнаул
1,алтайский край,г,бийск,181678,алтайскийкрай,бийск
2,алтайский край,г,рубцовск,124687,алтайскийкрай,рубцовск
3,алтайский край,г,новоалтайск,73122,алтайскийкрай,новоалтайск
4,алтайский край,г,заринск,40626,алтайскийкрай,заринск
...,...,...,...,...,...,...
2280,ярославская область,пгт,бурмакино,2554,ярославскаяобласть,бурмакино
2281,ярославская область,пгт,лесная поляна,2460,ярославскаяобласть,леснаяполяна
2282,ярославская область,пгт,поречье-рыбное,1405,ярославскаяобласть,поречьерыбное
2283,ярославская область,пгт,красный профинтерн,1076,ярославскаяобласть,красныйпрофинтерн


In [20]:
df_rus_popcity = df_rus_pop.merge(df_russtat, on=["reg_merge", "area_merge"], how="outer")
df_rus_popcity.to_csv("C:/work/gkgn/rus_citypop.csv", sep=";", encoding="utf-8-sig", index=False)

In [12]:
rus_city_path = "C:/work/gkgn/rus_citypop.xlsx"
df_rus_city = pd.read_excel(rus_city_path, sheet_name="rus", index_col=None, dtype="str")
df_rus_city["latitude_g"] = df_rus_city["latitude_g"].astype("float")
df_rus_city["longitude_g"] = df_rus_city["longitude_g"].astype("float")
df_rus_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156953 entries, 0 to 156952
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   region           156953 non-null  object 
 1   municipality     156953 non-null  object 
 2   type             156953 non-null  object 
 3   settelment       156930 non-null  object 
 4   latitude         156953 non-null  object 
 5   longitude        156953 non-null  object 
 6   latitude_g       156953 non-null  float64
 7   longitude_g      156953 non-null  float64
 8   city_population  2901 non-null    object 
dtypes: float64(2), object(7)
memory usage: 10.8+ MB


In [14]:
# антонов огонь
df_big_pop = pd.read_csv("C:/work/scripts/справочники/big_population.csv", sep=";", index_col=None)
df_big_pop

,id,region,municipality,settlement,type,population,children,latitude_dms,longitude_dms,latitude_dd,longitude_dd,oktmo,dadata,rosstat
0,0,Орловская область,Болховский,Колонтаева,д,0,0,53.22.07,035.54.36,53.368611,35.910000,5.460442e+10,0,1
1,1,Республика Крым,Алушта,Пушкино,с,273,0,44.35.45,034.20.27,44.595833,34.340833,3.570300e+10,0,1
2,2,Липецкая область,Лев-Толстовский район,Барятино,с,7,1,53.15.46,039.30.14,53.262778,39.503889,4.263641e+10,0,1
3,3,Тверская область,Селижаровский район,Хилово,д,2,0,56.54.20,033.25.09,56.905556,33.419167,2.865043e+10,0,1
4,4,Томская область,Парабельский район,Басмасово,д,6,0,58.38.12,082.02.40,58.636667,82.044444,6.964444e+10,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155916,155916,Ульяновская область,Вешкаймский район,Ребровка,д,0,0,53.59.12,046.48.35,53.986667,46.809722,7.360744e+10,0,1
155917,155917,Псковская область,Палкинский район,Шадрица,д,1,0,57.29.03,028.08.48,57.484167,28.146667,5.863742e+10,0,1
155918,155918,Псковская область,Себежский район,Овчинниково,д,0,0,56.19.48,028.26.17,56.330000,28.438056,5.865410e+10,0,1
155919,155919,Тульская область,рабочий пос. Новогуровский,Новогуровский,рп,3398,305,54.28.03,037.20.19,54.467500,37.338611,7.070200e+10,1,0


In [15]:
df_rus_city["reg_s"] = df_rus_city["region"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_rus_city["set_s"] = df_rus_city["settelment"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_rus_city["region_found"] = ""
df_rus_city["municipality_found"] = ""
df_rus_city["settlement_found"] = ""
df_rus_city["settelment_type_found"] = ""
df_rus_city["population_found"] = ""
df_rus_city["children_found"] = ""
df_rus_city["D_km_found"] = ""
df_rus_city["sf/e"] = ""

df_big_pop["reg_s"] = df_big_pop["region"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_big_pop["set_s"] = df_big_pop["settlement"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')

In [17]:
df_rus_city

,region,municipality,type,settelment,latitude,longitude,latitude_g,longitude_g,city_population,reg_s,set_s,region_found,municipality_found,settlement_found,settelment_type_found,population_found,children_found,D_km_found,sf/e
0,Алтайский край,Алейский район,посёлок,Алейский,52°28.2',82°43.7',52.470000,82.728333,NaN,алтайскийкрай,алейский,,,,,,,,
1,Алтайский край,Алейский район,посёлок,Александровский,52°23.2',82°55.4',52.386667,82.923333,NaN,алтайскийкрай,александровский,,,,,,,,
2,Алтайский край,Алейский район,село,Безголосово,52°31',82°58',52.516667,82.966667,NaN,алтайскийкрай,безголосово,,,,,,,,
3,Алтайский край,Алейский район,посёлок,Березовский,52°34.1',82°18.5',52.568333,82.308333,NaN,алтайскийкрай,березовский,,,,,,,,
4,Алтайский край,Волчихинский район,посёлок,Березовский,52°04.6',80°06.5',52.076667,80.108333,NaN,алтайскийкрай,березовский,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156948,Ярославская область,Ярославский район,деревня,Ямино,57°45',39°41.7',57.750000,39.695000,NaN,ярославскаяобласть,ямино,,,,,,,,
156949,Ярославская область,Ярославский район,деревня,Ямищи,57°31.8',39°53.9',57.530000,39.898333,NaN,ярославскаяобласть,ямищи,,,,,,,,
156950,Ярославская область,Ярославский район,посёлок,Ярославка,57°44.5',39°50.6',57.741667,39.843333,NaN,ярославскаяобласть,ярославка,,,,,,,,
156951,Ярославская область,Ярославский район,деревня,Ясино,57°54.6',39°59.8',57.910000,39.996667,NaN,ярославскаяобласть,ясино,,,,,,,,


In [19]:
%%time

# используем аццкий код-3, поиск с учётом названия населённого пункта, а если такого нет - только по координатам, статус поиска и совпадение букв

n = 0
lim = 157000
for index, row in df_rus_city.iterrows():
    sf = ""
    try:
        if index < lim:
            D_km = 100500
            region = "no_data"
            municipality = "no_data"
            settlement = "no_data"
            settelment_type = "no_data"
            population = "no_data"
            children = "no_data"
            pos_0 = (row["latitude_g"], row["longitude_g"])
            df_tmp = df_big_pop[df_big_pop["reg_s"] == row["reg_s"]]
            if row["set_s"] in (df_tmp["set_s"].to_list()):
                sf = "1"
                k = 0
                for i, r in df_tmp.iterrows():
                    pos_tmp = (r["latitude_dd"], r["longitude_dd"])
                    if row["set_s"] == r["set_s"]:
                        k += 1
                        D_tmp = GD(pos_0, pos_tmp).km
                        if D_tmp < D_km:
                            region = r["region"]
                            municipality = r["municipality"]
                            settlement = r["settlement"]
                            settelment_type = r["type"]
                            population = r["population"]
                            children = r["children"]
                            D_km = D_tmp
                sf = sf + "_" + str(k)
            else:
                sf = "0"
                l = 0
                for i, r in df_tmp.iterrows():
                    pos_tmp = (r["latitude_dd"], r["longitude_dd"])
                    D_tmp = GD(pos_0, pos_tmp).km
                    l += 1
                    if D_tmp < D_km:
                        region = r["region"]
                        municipality = r["municipality"]
                        settlement = r["settlement"]
                        settelment_type = r["type"]
                        population = r["population"]
                        children = r["children"]
                        D_km = D_tmp
                # срвнение названий населённых пунктов
                sf = sf + "_" + str(l) + "_" + str(round(word_similarity(row["set_s"], r["set_s"]), 2))
            df_rus_city.at[index, "region_found"] = region
            df_rus_city.at[index, "municipality_found"] = municipality
            df_rus_city.at[index, "settlement_found"] = settlement
            df_rus_city.at[index, "settelment_type_found"] = settelment_type
            df_rus_city.at[index, "population_found"] = population
            df_rus_city.at[index, "children_found"] = children
            df_rus_city.at[index, "D_km_found"] = round(D_km, 3)
            df_rus_city.at[index, "sf/e"] = sf
            n += 1
            clear_output()
            print(f"обработана строка {n} из {df_rus_city.shape[0]} при лимите {lim}\t{round(n*100/df_rus_city.shape[0], 3)} %\t({round(n*100/lim, 3)} %)", flush=True)
        else:
            break
    except Exception as e:
        print(f"строка {n} ошибка {e}")
        df_rus_city.at[index, "sf/e"] = e
print("сохранение")
df_rus_city.to_csv("C:/Users/Admin/Desktop/RESULT_3.csv", sep=";", encoding="utf-8-sig", index=False)
print("результат сохранён")

обработана строка 16723 из 156953 при лимите 157000	10.655 %	(10.652 %)


: 

In [1]:
df_rus_city

NameError: name 'df_rus_city' is not defined